<a style="width: 100%; text-align: center" href="https://colab.research.google.com/github/emasri/trd/blob/master/td.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setup

In [ ]:

#@markdown

import enum
import string
import unicodedata
import os, time,  re, IPython, urllib
from google.colab import output, drive
from copy import deepcopy
from glob import glob

LT = True #@param {type:"boolean"}
Drive = True #@param {type:"boolean"}
Test_Drive = False #@param {type:"boolean"}

if Drive and not os.path.exists("/content/drive"):
  drive.mount('/content/drive')
  os.environ['TZ'] = 'Turkey'
  time.tzset()
  !python -m pip install --upgrade pip setuptools wheel &> /dev/null
  !pip install "ipywidgets>=7,<8" &> /dev/null
  
if Test_Drive:
  direct_url = "https://download.visualstudio.microsoft.com/download/pr/c4c86d02-a47b-4bd4-b73d-ec3be19e5245/76c673e22a120464c95f85bef342a361/aspnetcore-runtime-6.0.7-win-x64.exe" 
  output_dir = "/content/test" 
  !wget "$direct_url" -O "$output_dir"  
  !mv "/content/test" "/content/drive/My Drive"

def dump(obj, filter = True):
  for attr in dir(obj):
    if filter and attr.startswith("__"):
      continue
    print("              %s =  %r" % (attr, getattr(obj, attr)))

def dirsize(path, string = False):
  size = 0
  for path, dirs, files in os.walk(path):
      for f in files:
          fp = os.path.join(path, f)
          size += os.path.getsize(fp)
  if string:
    return data_str(size)
  else:
    return size

def filter_dirs(ps):
  for p in ps:
    if p[0] == '.':
      ps.remove(p)
  return ps

def filter_checks(ls):
  check = 'ipynb_checkpoints'
  fltr = [s for s in ls if contains(s, check)]
  for s in fltr:
    ls.remove(s)
  return ls

def print_files(drs, range_start , range_end):
  for i in range(range_start, range_end):
    print("   {0}-  {1}".format(i+1, drs[i])) 
    
def RoundTo1(n) :
  if n < 1 and (n - 0.9765625) >= 0 :
    return 1
  else: 
    return n

def data_str(byte_val):
  size_gb = ByteToGB(byte_val)
  if (size_gb >= 1) :
    return "{:.1f} GB".format(size_gb)
  else:
    size_mb = ByteToMB(byte_val)
    return "{:.1f} MB".format(size_mb)

def speed_str(byte_val):
  size_mb = ByteToMB(byte_val)
  if (size_mb >= 1) :
    return "{:.1f} MB".format(size_mb)
  else:
    return "{:.1f} KB".format(ByteToKB(byte_val))

ByteToGB = lambda n: RoundTo1(n / 1073741824)
ByteToMB = lambda n: RoundTo1(n/ 1048576)
ByteToKB = lambda n: RoundTo1(n/ 1024)
contains = lambda string, search: search.lower() in string.lower()
checkpoint = '.ipynb_checkpoints'

SM = "SM1" #@param ["", "SM1", "SM2", "SM3", "SM4", "SM4", "SM5", "SM6", "SM7"]
Bak1 = "SM2" #@param ["", "SM1", "SM2", "SM3", "SM4", "SM4", "SM5", "SM6", "SM7"]
Bak2 = "SM3" #@param ["", "SM1", "SM2", "SM3", "SM4", "SM4", "SM5", "SM6", "SM7"]
Bak3 = "SM4" #@param ["", "SM1", "SM2", "SM3", "SM4", "SM4", "SM5", "SM6", "SM7"]

if SM:
  Save_Base = f"/content/drive/Shareddrives/{SM}/Downloads"  
  Movies_Path = f"/content/drive/Shareddrives/{SM}/Library/Movies" 
  Shows_Path = f"/content/drive/Shareddrives/{SM}/Library/TV Shows"
  Music_Path = f"/content/drive/Shareddrives/{SM}/Library/Music"

paths = [Save_Base, Movies_Path, Shows_Path]
if Bak1:
  bakdl1 = f"/content/drive/Shareddrives/{Bak1}/Downloads"
  bakmv1 = f"/content/drive/Shareddrives/{Bak1}/Library/Movies"
  baktv1 = f"/content/drive/Shareddrives/{Bak1}/Library/TV Shows"
  bakmc1 = f"/content/drive/Shareddrives/{Bak1}/Library/Music"
  paths.extend((bakdl1, bakmv1, baktv1, bakmc1))

if Bak2:
  bakdl2 = f"/content/drive/Shareddrives/{Bak2}/Downloads"
  bakmv2 = f"/content/drive/Shareddrives/{Bak2}/Library/Movies"
  baktv2 = f"/content/drive/Shareddrives/{Bak2}/Library/TV Shows"
  bakmc2 = f"/content/drive/Shareddrives/{Bak2}/Library/Music"
  paths.extend((bakdl2, bakmv2, baktv2, bakmc2))

if Bak3:
  bakdl3 = f"/content/drive/Shareddrives/{Bak3}/Downloads"
  bakmv3 = f"/content/drive/Shareddrives/{Bak3}/Library/Movies"
  baktv3 = f"/content/drive/Shareddrives/{Bak3}/Library/TV Shows"
  bakmc3 = f"/content/drive/Shareddrives/{Bak3}/Library/Music"
  paths.extend((bakdl3, bakmv3, baktv3, bakmc3))

mv_baks = {Bak1: bakmv1, Bak2: bakmv2, Bak3: bakmv3}
tv_baks = {Bak1: baktv1, Bak2: baktv2, Bak3: baktv3}
mc_baks = {Bak1: bakmc1, Bak2: bakmc2, Bak3: bakmc3}
dl_baks = {Bak1: bakdl1, Bak2: bakdl2, Bak3: bakdl3}
baks = {"mv": mv_baks, "tv": tv_baks, "mc": mc_baks, "dl": dl_baks}

path_mv = lambda mv: Movies_Path + "/" + mv
path_tv = lambda tv: Shows_Path + "/" + tv
path_sn = lambda tv, sn: Shows_Path + "/" + tv + "/" + sn
path_eps = lambda tv, sn, file: Shows_Path + "/" + tv + "/" + sn + "/" + file
path_bk = lambda cat, bak, dir: baks[cat][bak] + "/" + dir if dir else baks[cat][bak]

def getbak(path, bak):
  return path.replace(SM, bak)

for path in paths:
  if not os.path.exists(path):
    os.makedirs(path)

def is_empty(path):  
  if os.path.exists(path):
    dircontent = os.listdir(path)
    if len(dircontent) == 0 or (len(dircontent) == 1 and dircontent[0] == checkpoint ):
        return True
    else:
        return False
  else:
    return False

download_paths = glob("/content/drive/Shareddrives/*/Downloads")
uncompleted_sources = []
for item in download_paths:
  p0 = item + "/" + ".uncomplete"
  if os.path.exists(p0):
    if is_empty(p0):
      !rm -r "$p0"
    else:
      uncompleted_sources.append(p0)
      
#from os import makedirs makedirs("/root/.config/rclone", exist_ok = True)

if not SM:
  Save_Base = "/content/Downloads"  #@param {type:"string"}
  Movies_Path = "/content/Downloads/Movies" #@param {type:"string"}
  Shows_Path = "/content/Downloads/TV Shows" #@param {type:"string"}
  os.makedirs(Save_Base, exist_ok = True)
  os.makedirs(Movies_Path, exist_ok = True)
  os.makedirs(Shows_Path, exist_ok = True)

Active_Downloads = 5 #@param {Type:"integer"}
Part = "1" #@param {Type:"string"}

drive_path = "" 
done_path = "/content/done"
drive_path = Save_Base + "/d-0" + Part
temp_save="/content/d-0" + Part

os.makedirs("/content/done", exist_ok = True)
os.makedirs(temp_save, exist_ok = True)
os.makedirs("/content/Added", exist_ok = True)

seasons = {"season1":"S01", "season2":"S02", "season3":"S03", "season4":"S04", "season5":"S05", "season6":"S06", "season7":"S07", "season8":"S08", "season9":"S09", "season10":"S10", "season11":"S11", "season12":"S12", "season13":"S13", "season14":"S14", "season01":"S01", "season02":"S02", "season03":"S03", "season04":"S04", "season05":"S05", "season06":"S06", "season07":"S07", "season08":"S08", "season09":"S09"}


season_re = r"season\d{1,2}"
season_re2 = r"S\d{1,2}|s\d{1,2}"
year_re = r"\b\d{4}" 
eps_re = r"\b[Ss]\d{1,2}[Ee]\d{1,2}\b"

skip_list = ['.txt',
              '.inf',
              '.info',
              '.nfo',
              'www.yts',
              'rarbg_do_not'
            ]

l337x_tv = "TV"
l337x_movie = "Movies"
l337x_apps = "Apps"
l337x_games = "Games"
l337x_seeders = "seeders/desc"
l337x_size_asc = "size/asc"
l337x_size_desc = "size/desc"
l337x_time_asc = "time/asc"
l337x_time_desc = "time/desc"
    
new_trackers = ["udp://tracker.opentrackr.org:1337/announce", "udp://tracker.internetwarriors.net:1337/announce", "udp://exodus.desync.com:6969/announce", "udp://tracker.cyberia.is:6969/announce", "udp://3rt.tace.ru:60889/announce", "http://5rt.tace.ru:60889/announce", "udp://explodie.org:6969/announce", "udp://47.ip-51-68-199.eu:6969/announce", "udp://opentracker.i2p.rocks:6969/announce", "http://open.acgnxtracker.com:80/announce", "udp://www.torrent.eu.org:451/announce", "udp://tracker.torrent.eu.org:451/announce", "udp://tracker.tiny-vps.com:6969/announce", "udp://tracker.ds.is:6969/announce", "udp://retracker.lanta-net.ru:2710/announce", "udp://open.stealth.si:80/announce", "udp://tracker.moeking.me:6969/announce", "udp://ipv4.tracker.harry.lu:80/announce", "http://tracker.dler.org:6969/announce", "http://rt.tace.ru:80/announce", "udp://93.158.213.92:1337/announce", "udp://138.255.103.83:1337/announce", "udp://208.83.20.20:6969/announce", "udp://193.218.118.220:6969/announce", "udp://79.111.118.59:60889/announce", "http://93.157.234.32:60889/announce", "udp://184.105.151.164:6969/announce", "udp://51.68.199.47:6969/announce", "udp://51.81.46.170:6969/announce", "udp://91.216.110.52:451/announce", "udp://89.234.156.205:451/announce", "udp://5.206.60.196:6969/announce", "udp://5.226.148.20:6969/announce", "udp://37.235.174.46:2710/announce", "udp://185.181.60.67:80/announce", "udp://138.201.150.56:6969/announce", "udp://51.15.40.114:80/announce", "http://168.119.237.9:6969/announce", "http://195.201.31.194:80/announce", "https://trakx.herokuapp.com:443/announce", "https://w.wwwww.wtf:443/announce", "https://tracker.tamersunion.org:443/announce", "https://tracker.sloppyta.co:443/announce", "https://tracker.renfei.net:443/announce", "https://tracker.nitrix.me:443/announce",
              "https://tracker.nanoha.org:443/announce", "https://tracker.lelux.fi:443/announce", "https://tracker.imgoingto.icu:443/announce", "https://tracker.hama3.net:443/announce", "https://tracker.foreverpirates.co:443/announce", "http://tracker2.wasabii.com.tw:6969/announce", "udp://tracker.sktorrent.net:6969/announce", "http://www.wareztorrent.com:80/announce", "udp://bt.xxx-tracker.com:2710/announce", "udp://tracker.eddie4.nl:6969/announce", "udp://tracker.grepler.com:6969/announce", "udp://tracker.mg64.net:2710/announce", "udp://wambo.club:1337/announce", "udp://tracker.dutchtracking.com:6969/announce", "udp://tc.animereactor.ru:8082/announce", "udp://tracker.justseed.it:1337/announce", "udp://tracker.leechers-paradise.org:6969/announce", "https://open.kickasstracker.com:443/announce", "udp://tracker.coppersurfer.tk:6969/announce", "http://87.253.152.137/announce", "http://91.216.110.47/announce", "http://91.217.91.21:3218/announce", "http://91.218.230.81:6969/announce", "http://93.92.64.5/announce", "http://atrack.pow7.com/announce", "http://bt.henbt.com:2710/announce", "http://bt.pusacg.org:8080/announce", "https://tracker.bt-hash.com:443/announce", "udp://tracker.leechers-paradise.org:6969", "https://182.176.139.129:6969/announce", "udp://zephir.monocul.us:6969/announce", "https://tracker.dutchtracking.com:80/announce", "https://grifon.info:80/announce", "udp://tracker.kicks-ass.net:80/announce", "udp://p4p.arenabg.com:1337/announce", "udp://tracker.aletorrenty.pl:2710/announce", "https://tracker.parrotsec.org:443/announce", "https://tracker.moxing.party:6969/announce", "https://tracker.ipv6tracker.ru:80/announce", "https://tracker.fastdownload.xyz:443/announce", "https://gwp2-v19.rinet.ru:80/announce", "https://tr.kxmp.cf:80/announce", "https://explodie.org:6969/announce"]


class Types (enum.Enum):
  General = 1 
  Movie = 2
  Show = 3
  Season = 4
  Episode = 5

class attrs:
  backup = False
  completed = False
  replace = False
  name = ""
  save_name = ""
  save_path = ""
  imdb_name = ""
  type_ = ""
  sequential = False
  download_size = 0
  uncomplete_source = ""
  show = ""
  season = ""
  existing_dir = ""
  existing_path = ""
  existing_file = ""
  baksv1 = ""
  baksv2 = ""
  baksv3 = ""
  year = 0
  imdb_rating = ""
  imdb_votes = ""
  quality = "1080"
  category = ""
  site_source = ""
  uploader = ""
  format = ""
  duration = ""
  video_width = ""
  video_height = ""
  frame_rate = ""
  bit_rate = ""
  color_bit_depth = ""
  video_bit_rate = ""
  audio_tracks = None

class imdb_params:
  def __init__(self, keyword, title_type, s_year, e_year, ratings, min_votes, max_run, lists, labels, imdb_sort, 
    filter_watched: bool, filter_downloaded, l337x_download, rarbg_download, site_category, download_params, l337x_sort, rarbg_auto):
    self.keyword = keyword
    self.title_type = title_type
    self.s_year = s_year
    self.e_year = e_year
    self.ratings = ratings
    self.min_votes = min_votes
    self.max_run = max_run
    self.lists = lists
    self.labels = labels
    self.imdb_sort = imdb_sort
    self.filter_watched = filter_watched
    self.filter_downloaded = filter_downloaded
    self.l337x_download = l337x_download
    self.rarbg_download = rarbg_download
    self.site_category = site_category
    self.download_params = download_params
    self.l337x_sort = l337x_sort
    self.rarbg_auto = rarbg_auto

  keyword = ""
  title_type = ""
  s_year = ""
  e_year = ""
  ratings = ""
  min_votes = ""
  max_run = ""
  lists = ""
  labels = ""
  imdb_sort = ""
  filter_watched = False
  filter_downloaded = False
  l337x_download = False
  rarbg_download = False
  site_category = ""
  download_params = ""
  l337x_sort = l337x_seeders
  rarbg_auto = False

class imdb_result:
  def __init__(self, title, runtime, genres, rating, votes, director, staring):
    self.title = title
    self.runtime = runtime
    self.rating = rating
    self.votes = votes
    self.director = director
    self.staring = staring
    self.genres = genres

class l337x_result:
  def __init__(this, title, size = 0, date = "", seeders = "", leechers = "", uploader = "", href = "", state = "" ):
    this.title = title
    this.size = size
    this.date = date 
    this.seeders = seeders
    this.leechers = leechers
    this.uploader = uploader
    this.href = href
    this.state = state

attr0 = attrs()

def transtitle(name): 
  name = name.replace("&","and")
  dgts = re.findall(year_re, name)
  if dgts: 
    if len(dgts) == 1:
      name = name.split(dgts[0])[0]  + dgts[0]
    else:
      name = name.split(dgts[-1])[0] + dgts[-1]
  table = name.maketrans("",""," '(#$%-,.;:!?)")
  return name.translate(table).lower()

def show_name_trans(name):
  name = name.split('1080')[0]
  dgts = re.findall(year_re, name)
  if dgts:
    name = name.split(dgts[0])[0].lower()
  else :
    name = name.lower()
  table = name.maketrans("",""," '(#$%&-,.;:!?)")
  trans = name.translate(table)
  return trans

def movie_exists(name):
  movies_raw = filter_dirs(next(os.walk(Movies_Path))[1])
  name_s = transtitle(name)
  for i in range(len(movies_raw)):
    path = path_mv(movies_raw[i])
    if name_s == transtitle(movies_raw[i]) and dirsize(path) > 1000000:
      return (movies_raw[i], path_mv(movies_raw[i]))
  return False

def show_exists(name):
  shows_raw = filter_dirs(next(os.walk(Shows_Path))[1])
  name_s = transtitle(name)
  for i in range(len(shows_raw)):
    path = path_tv(shows_raw[i])
    if name_s == transtitle(shows_raw[i]) and not is_empty(path):
      return (shows_raw[i], path_tv(shows_raw[i]))
  return ""

def season_exists(show, season):
  search = show_exists(show)
  if search:
    show = search
    path = path_tv(show) + "/" + season
    if os.path.exists(path) and not is_empty(path):
      return (show, path)
  return ""

def episode_exists(show, season, name):
  search = show_exists(show)
  if search:
    show = search
    checkpath = path_sn(show, season)
    if os.path.exists(checkpath) and not is_empty(checkpath):
      check = re.search(eps_re, name)
      eps = check.group() if check else name
      epss = next(os.walk(checkpath))[2]
      for e in epss:
        splt = os.path.splitext(e)
        if splt[1] == ".mp4" or splt[1] == ".mkv": 
          if contains(splt[0], eps):
            return (e, show, path_eps(show, season, e))
  return ""

def movie_file(path, name):
  if path and os.path.exists(path):
    files = next(os.walk(path))[2]
    for file in files:
      splt = os.path.splitext(file)
      ext = splt[1]
      title: str = splt[0].lower()
      if (ext == ".mp4" or ext == ".mkv" or ext == ".av" or ext == ".ts") and name.lower().startswith(title):
        return path + "/" + file
  return ""

def replace_download(path):
  if path and os.path.exists(path):
    parent = os.path.dirname(path)
    rp = parent + "/" + ".replace"
    os.makedirs(rp, exist_ok=True)
    !mv "$path" "$rp"

def get_magnet_name(magnet):
  return urllib.parse.unquote(magnet.split("dn=")[1].split("&tr=")[0], encoding='utf-8', errors='replace').replace('+',".")

def prepare_download(attr: attrs):
  Type = attr.type_
  if Type == Types.Movie: 
    newname = name_movie(attr.name, attr.quality)
    search = movie_exists(newname)
    if search:
      path = search[1]
      if not attr.replace:
        size = dirsize(path, True)
        print(f"Warning: Prepare Download: The movie {search[0]} is already downloaded. Size: {size}. Do you want to overwrite?")
        if input() == "n":
          return -1
      attr.existing_dir = search[0]
      attr.existing_path = path
      attr.existing_file = movie_file(path, search[0])
      if attr.replace or not attr.save_name:
        attr.save_name = search[0]
    if not attr.save_name:
      attr.save_name = newname          
    attr.save_path = Movies_Path
    if attr.backup:
      if Bak1: attr.baksv1 = bakmv1
      if Bak2: attr.baksv2 = bakmv2
      if Bak3: attr.baksv3 = bakmv3       
      
  elif Type == Types.Show:
    newname = name_show(attr.name, attr.quality)
    search = show_exists(newname)
    if search:
      path = search[1]
      if not attr.replace:
        print(f"Warning: Prepare Download: The show {search[0]} is already downloaded. Do you want to overwrite?")
        if input() == "n":
          return -1
      attr.existing_dir = search [0]   
      attr.existing_path = path
      if attr.replace or not attr.save_name:
        attr.save_name = search[0]
    if not attr.save_name:
      attr.save_name = newname
    attr.save_path = Shows_Path
    if attr.backup:
      if Bak1: attr.baksv1 = baktv1
      if Bak2: attr.baksv2 = baktv2
      if Bak3: attr.baksv3 = baktv3

  elif Type == Types.Season:
    newname = name_season(attr.name)
    show = name_show(attr.name, attr.quality)
    search = season_exists(show, newname)
    if search:
      show = search[0]
      path = search[1]
      if not attr.replace:
        size = dirsize(path, True)
        print(f"Warning: Prepare Download: {show} {newname} is already downloaded. Size: {size}. Do you want to overwrite?")
        if input() == "n":
          return -1
      attr.existing_path = path
    attr.show = show
    attr.save_path = path_tv(show)
    os.makedirs(attr.save_path, exist_ok= True)
    if not attr.save_name:
      attr.save_name = newname
    if attr.backup:  
      if Bak1:
        attr.baksv1 = path_bk("tv", Bak1, show)
        os.makedirs(attr.baksv1, exist_ok= True)
      if Bak2: 
        attr.baksv2 = path_bk("tv", Bak2, show)
        os.makedirs(attr.baksv2, exist_ok= True)
      if Bak3: 
        attr.baksv3 = path_bk("tv", Bak3, show)
        os.makedirs(attr.baksv3, exist_ok= True)

  elif Type == Types.Episode:
    newname = name_episode(attr.name, attr.quality)
    show = name_show(attr.name)
    season = name_season(attr.name)
    search = episode_exists(show, season, newname)
    if search:
      show = search[1]
      path = search[2]
      size = data_str(os.path.getsize(path))
      print(f"Warning: Prepare Download: {newname} is already downloaded. Size: {size}. Do you want to overwrite?")
      if input() == "n":
        return -1
      attr.existing_path = path
      attr.existing_file = search[0]
    attr.show = show
    attr.season = season
    attr.save_path = path_sn(show, season)
    os.makedirs(attr.save_path, exist_ok= True)
    if not attr.save_name:
      attr.save_name = newname
    if attr.backup:
      edir = f"{show}/{season}"
      if Bak1:
        attr.baksv1 =  path_bk("tv", Bak1, edir)  
        os.makedirs(attr.baksv1, exist_ok= True)
      if Bak2: 
        attr.baksv2 = path_bk("tv", Bak2, edir)  
        os.makedirs(attr.baksv2, exist_ok= True)
      if Bak3: 
        attr.baksv3 = path_bk("tv", Bak3, edir)  
        os.makedirs(attr.baksv3, exist_ok= True)

  else:
    attr.save_path = drive_path
    if attr.backup:
      if Bak1: attr.baksv1 = bakdl1
      if Bak2: attr.baksv2 = bakdl2
      if Bak3: attr.baksv3 = bakdl3
  return 0        

def add_local_files(attr_: attrs = attr0):
  fls = next(os.walk("."))[2]
  for fl in fls:
    attr = deepcopy(attr_)
    splited = os.path.splitext(fl)
    if splited[1] == ".torrent":
      ti = lt.torrent_info(fl)
      attr.name = ti.name()
      if not ti.is_valid():
        print(f"Invalid torrent:{attr.name}")
        continue
      if prepare_download(attr) == 0:
        add_torrent_file(ti, attr)
        !mv "$fl" "/content/Added"

def add_torrent_file(ti, attr: attrs):
  print("Downloading: " + attr.name)
  params = {
  "save_path": temp_save,
  "ti": ti}
  print("Files: {0}".format(ti.num_files()))
  print("Metadata size: {0}".format(speed_str(ti.metadata_size())))
  print("Download files:\n")
  files = ti.files()
  handle = ses.add_torrent(params)
  key = str(handle.info_hash())

  if key in downloads_dict:
    print(" Error: add_torrent_file: Torrent already added.\n")
    return
  handle.auto_managed(True)
  handle.completed = False
  handle.attr = attr
  downloads_dict[key] = handle
  active.append(handle)
  add_trackers(handle)
  for index in range(files.num_files()):
    if any(contains(files.file_name(index), check) for check in skip_list):
      handle.file_priority(index,0)
    else: 
      handle.file_priority(index,6)
      fs_str= data_str(files.file_size(index))
      fi_str = "   {:3d}-  ".format(index) + fs_str
      print(fi_str, files.file_name(index))
  print()
  
def add_torrent_magnet(magnet_link, attr: attrs):
  params = {"save_path": temp_save}
  handle = lt.add_magnet_uri(ses, magnet_link, params)    
  if not handle.is_valid():
    print(" Invalid torrent  {0}".format(attr.name))
    return False
  key = str(handle.info_hash())
  if key in downloads_dict:
    print(" Error:  Add_Torrent_Magnet:  Torrent already added.\n")
    return False
  prepare_download(attr)
  handle.attr = attr
  handle.auto_managed(True)
  handle.set_sequential_download(attr.sequential)  
  handle.completed = False
  downloads_dict[key] = handle
  active.append(handle)
  add_trackers(handle)
  if handle.has_metadata():
    ti = handle.get_torrent_info()
    files = ti.files()
    print()
    for index in range(files.num_files()):
      if any(contains(files.file_name(index), check) for check in skip_list):
        handle.file_priority(index,0)
      else: 
        handle.file_priority(index,6)
        fs_str= data_str(files.file_size(index))
        fi_str = "   {:3d}-  ".format(index) + fs_str
        print(fi_str, files.file_name(index))
  print()
  return True

def add_trackers(handle):
  last_tier = handle.trackers()[-1]['tier']
  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': last_tier}
    handle.add_tracker(annouce_entry) 

def filterfiles():
  for h in active:
    if(h.has_metadata()):
      torinfo = h.get_torrent_info()
      files = torinfo.files()
      for x in range(files.num_files()):
        if any(contains(files.file_name(x), check) for check in skip_list):
          h.file_priority(x,0)
        else: 
          h.file_priority(x,5)
  total_download = 0
  print("Download files:")
  for h in active:
    print("")
    print("  "+h.name())
    if(h.has_metadata()):
      priorities = h.file_priorities()
      torinfo = h.get_torrent_info()
      files = torinfo.files()
      status= h.status()
      file_sizes = status.total_wanted
      total_download += file_sizes
      size_str = data_str(file_sizes)
      print("  Total download size: {}\n".format(size_str))
      print("  Index Size    Name")
      fi_str=""
      for x in range(files.num_files()):
        if (priorities[x]>0):
          fs_str= data_str(files.file_size(x))
          fi_str = "   {:3d}-  ".format(x) + fs_str
          print(fi_str,files.file_name(x))
    else:
      print("  Metadata not recieved yet.\n")
  all_total_str = data_str(total_download)
  print("Total download: ", all_total_str)

def get_year(name): 
  dgts = re.findall(year_re, name.split('1080')[0])
  if dgts: 
    return dgts[-1]
  else:
    return this_year

def name_movie(name, quality = "1080"): 
  name = name.split(quality)[0]
  table = name.maketrans("."," ","()")
  trans = name.translate(table)
  dgts = re.findall(year_re, trans)
  if dgts and len(dgts) >= 1:
       return trans.split(dgts[-1])[0] + dgts[-1]
  else:
    return trans.strip()

from datetime import date
this_year = date.today().year
def name_show(name, quality = "1080"): 
  name = name.split(quality)[0]
  table = name.maketrans("."," ","'()")
  trans = name.translate(table)
  dgts = re.findall(year_re, trans)
  if dgts: 
    return trans.split(dgts[-1])[0] + dgts[-1]
  elif re.search(season_re2, trans) != None: 
    return f"{re.split(season_re2, trans)[0]}{this_year}"
  else: 
    return f"{trans} {this_year}"

def name_season(name):
  table = name.maketrans("",""," .-_")
  trans = name.translate(table).lower()
  checkname = re.search(season_re, trans)
  if checkname:
    return seasons[checkname.group()].capitalize()
  checkname = re.search(season_re2, trans)
  if checkname:
    return checkname.group().capitalize()
  return get_year(name)

def name_episode(name, quality = "1080", keepChar = ""):
  if quality in name:
    name = name.split("1080")[0]
  elif "720p" in name:
    name = name.split("720p")[0]
  if keepChar: 
    r = s = ""
  else:
    r ="."
    s = " "
  table = name.maketrans(r, s, "()")
  name = name.translate(table)
  if name[-1] == ".":
    name = name[:-1]
  return name.strip()

def shave (handle, path, maindir) :
  ti = handle.get_torrent_info()
  files = ti.files()
  priorities = handle.file_priorities()
  for i in range(len(priorities)):
    file_path = path + "/" + files.file_path(i)
    if os.path.isfile(file_path) and priorities[i] == 0:
      !rm "$file_path"
      dirpath = os.path.dirname(file_path)
      if is_empty(dirpath):
        !rm -r "$dirpath"
  dircontent = next(os.walk(maindir))[1]
  for path in dircontent:
    if is_empty(path):
      !rm -r "$dirpath"

def after_shave (path, newname):
  dircontent = next(os.walk(path))[2]
  srt = 0
  for file in dircontent:
    split = os.path.splitext(file)
    ext = split[1]
    p1 = path + "/" + file
    if  any(contains(file, check) for check in skip_list):
      !rm "$p1"
    elif  (ext == ".mp4" or ext == ".mkv" or ext == ".srt") and not split[0] == newname:
      extra = ""
      if ext == ".srt":
        if srt > 0 :
          extra = str(srt)
        srt += 1
      p2 = path + "/" + newname + extra + ext
      !mv "$p1" "$p2"

  subspath = path + "/" + "Subs"
  if not os.path.exists(subspath):
    subspath = path + "/" + "subs"
    if not os.path.exists(subspath):
       return
  
  subs =  next(os.walk(subspath))[2]
  count = 0
  extra = ""
  for sub in subs:
    split = os.path.splitext(sub)
    ext = split[1]
    subname = newname + extra
    if ext == '.srt' and split[0] != subname:
      if count > 0:
        extra = str(count)
      p1 = subspath + "/" + sub
      p2 = subspath + "/" + subname + ext
      !mv "$p1" "$p2"
      count+=1

def clean_movies_dir(movie_path = Movies_Path) :
  dir_content = filter_checks(next(os.walk(movie_path))[1])
  cleanpath = ""
  for dir in dir_content :
    cleanpath = movie_path + "/" + dir
    clean_movie(cleanpath, dir, movie_path)
    
def clean_movie(path, name, movie_path = Movies_Path):
  newname = name
  search = re.search(r"\d{4}$", name) #Title ends with year.
  if not search:
    newname = name_movie(name)
    p1 = path
    p2 = movie_path + "/" + newname
    !mv "$p1" "$p2"
    path = p2
    print(newname)
  after_shave(path, newname)

def clean_shows(path = ""):
  if not path:
    path = Shows_Path
  shows = next(os.walk(path))[1]
  for show in shows:
    print(show)
    path = path + "/" + show
    clean_tv(path)

def clean_tv (path, keepChar = ""):
  dircontent = next(os.walk(path))[1]
  if len(dircontent) != 0:
    for season in dircontent:
      check = season.lower()
      if len(season) > 3 and ("season" in check or "s0" in check):
        newname = name_season(season)
        if newname and newname != season:
          p0 =  path + "/" + season
          p1 = path + "/" + newname
          !mv "$p0" "$p1"
          season = newname
      seasonpath = path + "/" + season
      clean_season(seasonpath, keepChar)
  else:
    clean_season(path, keepChar)

def clean_season(path, keepChar = ""):
  if not os.path.exists(path):
    return
  files = next(os.walk(path))[2]
  for file in files:
    splitted = os.path.splitext(file)
    eps = splitted[0]
    ext = splitted[1]
    p1 = path + "/" + file
    if ext == ".mkv" or ext == ".mp4" or ext == ".srt":
      newname = name_episode(eps, keepChar) + ext
      if newname != file:
        p2 = path + "/" + newname
        !mv "$p1" "$p2" 
    elif any(contains(file.lower(), check) for check in skip_list):
      !rm "$p1" 

def clean_download(type_, path, newname):
  if type_ == Types.Movie:
    after_shave(path, newname)
  elif type_ == Types.Show:
    clean_tv(path)
  elif type_ == Types.Season or type_ == Types.Episode:
    clean_season(path)

def normalize(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def search_title(title_raw, year_raw):
  year = re.search(year_re, year_raw).group()
  title = "{0} {1}".format(title_raw, year)
  table = title.maketrans("/"," ",":'(,?)")
  title = title.translate(table)
  return normalize(title.replace("&", "and"))

def scrap_watched_list(text):
  soup = BeautifulSoup(text,'html.parser')    
  scraped = soup.find("div",{"class":"lister-list"})
  if not scraped :
      return
  all_items = scraped.findAll(class_="lister-item-header")
  for i in range(len(all_items)):
    year_raw = all_items[i].find(class_="lister-item-year").text
    title_raw = all_items[i].a.text
    title = search_title(title_raw, year_raw)      
    watched_movies_list.append(title)
  return

def AdvancedIMDBSearch ( params: imdb_params, attr_ : attrs):
  url=f'https://www.imdb.com/search/title/?title={params.keyword}&title_type={params.title_type}&release_date={params.s_year}-01-01,{params.e_year}-12-31&user_rating={params.ratings}&num_votes={params.min_votes},&runtime=,{params.max_run}&my_ratings=exclude&lists={params.lists}&groups={params.labels}&sort={params.imdb_sort}&view=advanced&count=250'
  resp=requests.get(url, headers = {"Accept-Language": "en-US"})
  if resp.status_code == 200:
      print("\n IMDB Search Done Successfully...\n")
      soup = BeautifulSoup(resp.text,'html.parser')    
      l = soup.find("div", {"class":"lister-list"})
      if not l :
        print("AdvancedIMDBSearch: Error: An error occured while performing the web scrapping.")
        return
      movie_list = []
      downloaded= []
      all_items = l.findAll(class_="lister-item")
      shift = 0
      for i in range(len(all_items)):
        year_raw = all_items[i].find(class_="lister-item-year").text
        title_raw = all_items[i].find(class_="lister-item-header").a.text
        title = search_title(title_raw, year_raw)
        runtime = all_items[i].find(class_="runtime").text
        genres = all_items[i].find(class_="genre").text.strip()
        rating = all_items[i].find(class_="ratings-imdb-rating")["data-value"]
        votes = all_items[i].find("span", {"name":"nv"}).text
        res= imdb_result(title, runtime, genres, rating, votes, "", "")
        if title in watched_movies_list and params.filter_watched:
          continue
        if movie_exists(title):
          downloaded.append(res)
          if params.filter_downloaded:
            continue         
        movie_list.append(res)
      print()
      watch_state=""
      if params.filter_downloaded:
        for item in downloaded:
          if item.title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("        {:<70}{:<30}DOWNLOADED{:>10}{:^12}{:15}{:>10}".format(item.title, item.genres, item.runtime, item.rating, item.votes, watch_state))
        for index in range(len(movie_list)):
          if movie_list[index].title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("  {:>3} - {:<70}{:<29}{:>21}{:^12}{:15}{:>10}".format(index + 1 , movie_list[index].title, movie_list[index].genres, movie_list[index].runtime, movie_list[index].rating, movie_list[index].votes, watch_state))
      else:
        watch_state = download_state = ""
        for index in range(len(movie_list)):          
          if movie_list[index].title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""          
          if movie_list[index] in downloaded:
            download_state = "DOWNLOADED"
          else:
            download_state = ""
          print("  {:>3} - {:<70}{:<29}{:>15}{:>15}{:^12}{:15}{:>10}".format(index + 1,  movie_list[index].title, movie_list[index].genres,  download_state, movie_list[index].runtime, movie_list[index].rating, movie_list[index].votes, watch_state))
      
      if params.rarbg_download or params.l337x_download:
        print("\n--> Choose movie range to download:       Format: [start-end]      [title1,title5..]     [single]     Exit: enter \n")
        start = end = -1
        rng = input().lower()
        if rng == "" or rng == 'q' or rng == "s" or rng == " ":
          return
        elif "," in rng:
          rngs = rng.split(',')
          for item in rngs:
            if "r" in item:
              try:
                rename_title(movie_list[int(item.replace("r", ""))-1].title)
              except Exception as e:
                print(e)
              continue
            if "-" in item:
              in_rngs = item.split('-')
              start = int(in_rngs[0])-1
              end = int((in_rngs[1]))
              for i in range(start, end):
                attr = deepcopy(attr_)
                attr.save_name = movie_list[i].title             
                if params.l337x_download and params.rarbg_download:         
                  print("Downloading: {}\nSelect source:\n  1- 1337x\n  2- rarbg\n".format(movie_list[i].title))
                  select= input("").lower()
                  if select == "s" or select == "" : 
                    continue
                  if select == "q" or select == " " : 
                    return
                  select= int(select)
                  if select == 1:                  
                    search("1337x", f"{movie_list[i].title} {params.download_params}",  params.site_category, attr = attr, sort1337x = params.l337x_sort)
                  else: 
                    search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
                elif params.l337x_download:
                    search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
                elif params.rarbg_download:    
                    search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
            else:
              i = int(item)-1
              attr = deepcopy(attr_)
              attr.save_name = movie_list[i].title                
              if params.l337x_download and params.rarbg_download:
                print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[i].title))
                select= input("").lower()
                if select == "s" or select == "": continue
                if select == "q" or select == " ": return
                select= int(select)
                if select == 1:
                  search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
                else:
                  search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
              elif params.l337x_download:
                search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
              elif params.rarbg_download:  
                search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)  

        elif "-" in rng:
          rngs = rng.split('-')
          start = int(rngs[0])-1
          end = int((rngs[1]))
          for i in range(start, end):
            attr = deepcopy(attr_)
            attr.save_name = movie_list[i].title            
            if params.l337x_download and params.rarbg_download:
                print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[i].title))
                select= input("").lower()
                if select == "s" or select == "" : continue
                if select == "q" or select == " ": return
                select= int(select)
                if select == 1:
                    search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
                else:
                    search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
            elif params.l337x_download:
              for i in range(start, end):
                search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
            elif params.rarbg_download:  
              for i in range(start, end):
                search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
        else:
          if "r" in rng:
            try:
              rename_title(movie_list[int(rng.replace("r", ""))-1].title)
            except Exception as e:
              print(e)
          else:
            try:
              i = int(rng) - 1
            except Exception as e:
              print(e)
            attr = deepcopy(attr_)
            attr.save_name = movie_list[i].title              
            if params.l337x_download and params.rarbg_download:
              print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[i].title))
              select= input("").lower()
              if select == "s" or select == "q" or select == "" or select == " ":
                return
              select= int(select)
              if select == 1:
                search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
              else:
                search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
            elif params.l337x_download:
              search("1337x", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, sort1337x = params.l337x_sort)
            elif params.rarbg_download:  
              search("rarbg", f"{movie_list[i].title} {params.download_params}", params.site_category, attr = attr, auto = params.rarbg_auto)
        ses.start_dht()
        print("\n         Done")
  else:
      print("AdvancedIMDBSearch: Error accessing the web page. Code error: {0}".format(resp.state_code))

def rename_title(newname):
  print(f"Renaming: {newname}\nEnter title name for search or proceed:")
  inp = input().lower()
  if inp:
    words = inp.split()
  else:
    words = newname.split()
  check1 = " ".join(words[:2])
  check2 = len(words[0]) > 3
  movies = filter_dirs(next(os.walk(Movies_Path))[1])
  res = []
  for mv in movies:
    if mv.startswith(check1) or (check2 and mv.startswith(words[0])) or any(len(check3) > 3 and contains(mv, check3) for check3 in words):
      res.append(mv)
  if len(res) == 1:
    title = res[0]
  elif res:
    res.sort()
    i = 0
    for mv in res:
      i += 1
      print(f"{i} {mv}")       
    print("\nChoose a title to rename:\n")
    c = input()
    if c == "" or c == " " or c == "q" or c == "s":
      print("No title was renamed.\n")
      return
    else:
      try:
        title = res[int(c)-1]
      except Exception as e:
        print(e)
        return
  else:
    print("No title was renamed.\n")
    return

  print(f"Renaming: {title} to {newname} Confirm?")
  if input() != "n":
    p0 = Movies_Path + "/" + title
    p1 = Movies_Path + "/" + newname
    print(f"Renaming: {p0}")
    !mv "$p0" "$p1"

    p0 = bakmv1 + "/" + title
    if Bak1 and os.path.exists(p0):
      p1 = bakmv1 + "/" + newname
      print(f"Renaming backup: {p0}")
      !mv "$p0" "$p1"

    p0 = bakmv2 + "/" + title
    if Bak2 and os.path.exists(p0):
      p1 = bakmv2 + "/" + newname
      print(f"Renaming backup: {p0}")
      !mv "$p0" "$p1"

    p0 = bakmv3 + "/" + title
    if Bak3 and os.path.exists(p0):
      p1 = bakmv3 + "/" + newname
      print(f"Renaming backup: {p0}")
      !mv "$p0" "$p1"
    print(".....Rename done......")
  else:
    print("No title was renamed.\n")

def search(Site, Search, category, attr, sort1337x = l337x_seeders, auto = True, Page = 1):
  if Site == "1337x":
    pagestate: int = max(Page, 1)
    while pagestate > 0 :
      pagestate = scrap1337x(Search, sort1337x, category, page = pagestate, attr = attr)
    if pagestate == -1:
      print("Do you want to repeat search with rarbg? any\n")
      choice = input()
      if choice == "y" or choice == " " or choice == "":
        DownloadRarbg(Search, category, auto, attr = attr)
  else:
    res = DownloadRarbg(Search, category, auto, attr = attr)
    if res == -1 :
      print("Do you want to repeat search with 1337x? any\n")
      choice = input()
      if choice == "y" or choice == " " or choice == "":
        page = 1
        while page > 0 :
          page = scrap1337x(Search, sort1337x, category,  page = Page, attr = attr)

def check_download(type0, name):
  if type0 == Types.Movie:
    name = name_movie(name)
    return movie_exists(name)
  if type0 == Types.Show:
    name = name_show(name)
    return show_exists(name)

def DownloadRarbg(search, category, auto = False, attr: attrs = attr0):
  print(f"\nSearching for:  {search}  in rarbg.com.\n")
  if category == "Movies":
    all_cat = rarbg_movie
    categories = [rarbg_movie_h265]
  elif category == "TV" : 
    all_cat = rarbg_tv
    categories = rarbg_tv
  else:
    all_cat = rarbg_all
    categories = []
  state = 1                                  # 1 init, 2 edit & redo search, 0 done, -1 bad search
  while state != 0 :
    if state == 1:
      results = rarbg.search(search_string = search, categories = categories)
    elif state == 2:
      print(f"\nEdit search, q to return: {search}\n")
      newinput = input()
      if len (newinput) < 4:
        return -1
      else:
        results = rarbg.search(search_string = newinput, categories = all_cat)
        state = 0
    if not results and categories and state != 2 :    
      print("Trying in all categories..\n")
      results = rarbg.search(search_string = search, categories = all_cat)
    if results: 
      print("\n     {:<70}{:>10}{:>10}\n".format("Title", "Size", "Seeders"))
      for i in range(len(results)):
        print(" {:<2}- {:<70}".format(i + 1 , results[i].filename))
      if len(results) == 1 and auto:
        choice = 0
      else:
        print("\nPlease choose one of the results to download             Format [single]     Enter q to exit, e to edit the search:")
        choice = -1
        while choice == -1:                   #out of range get input again.
          choice = input().lower()
          if choice == "" or choice == " " or choice == "s" or choice == "q": 
            return -1
          elif choice == "e":
            state = 2
            break
          choice = int(choice) - 1
          if choice < 0 or choice >= len(results):
            print("\nChoice out of range.")
            choice = -1
        if state == 2:
          continue
      print(f"\nAdding torrent: {results[choice].filename}")
      attr.site_source = "rarbg"
      attr.name = results[i].filename
      if add_torrent_magnet(results[choice].download, attr):
        print("...... Added ......\n")
        state = 0
      else:
        print(f"Search Rarbg: Error encourted while trying to add: {results[i].filename}")
        state = 3
    else:
      print("No results were found.\n\nEnter e to edit the search, q to exit:\n")
      choice = input().lower()
      if  choice == "" or choice == " " or choice == "s" or choice == "q": 
        return -1
      if choice == "e":  
        state = 2
  return 0

def scrap1337x(search, sort, category = "", page = 1, attr = attr0):
  search = search.replace("'", " ").replace(" ","%20")
  if category:
    parameters = f"sort-category-search/{search}/{category}/{sort}/{page}/"
  else:
    parameters = f"sort-search/{search}/{sort}/{page}/"
  url = "https://www.1337xx.to/" + parameters
  state = 1
  while state:
    if state == 1:
      print("Page: {} \n".format(url))
      response = requests.get(url, headers= {"Accept-Language": "en-US"})
    elif state == 2:
      print(f"Edit search, q to return: {search}")
      newinput = input()
      if len (newinput) < 4: return -1
      else:
        newurl = f"https://www.1337xx.to/sort-search/{newinput}/{sort}/1/"
        print("Page: {} \n".format(newurl))
        response = requests.get(newurl, headers= {"Accept-Language": "en-US"})
        state = 0
    if response.status_code == 200 :
      soup = BeautifulSoup(response.text, 'html.parser' )
      main_element = soup.find("tbody")
      if main_element is None:
        print("\nNull search. Enter e to edit the search, q to exit.\n")
        choice = input().lower()
        if choice == "e":
          state = 2
          continue
        else: return -1
      all_res = main_element.findAll("tr")
      title = size = uploader = seeders = leechers = ""
      results = [] 
      if all_res:
        print("\n    {:<120}{:^12}    {:^15}{:^21}{:^4}{:>14}\n".format("Title", "Size", "Date", "Uploader", "Seeders", "Leechers"))
      for i in  range(len(all_res)):
        alldata= all_res[i].findAll("td")
        title = alldata[0].findAll("a")[1].text
        size = alldata[4].text
        date = alldata[3].text
        seeders = alldata[1].text
        leechers = alldata[2].text
        uploader = alldata[5].a.text
        if check_download(attr.type_, title):
          state = "Downloaded"
        else:
          state = ""
        print("{:<2}- {:<118}{:>11}       {:<16}{:>12}{:>12}{:>13}{:>14}".format(i + 1, title.strip(), size, date, uploader, seeders, leechers, state))
        href = "https://www.1337xx.to" + alldata[0].findAll("a")[1]['href']
        res = l337x_result(title= title, size= size, date= date, seeders= seeders, leechers= leechers, uploader= uploader, href= href, state=state)
        results.append(res)

      print("\nPlease choose a result to download:\n        Enter space, s or q to quit, n for the next page, p for the previous page, p+page to choose page.\n")
      choice = input().lower()
      if choice == "" or choice == " " or choice == "s" or choice == "q":
        print("\nExiting..\n")
        return -1
      if choice == 'n':
        return page + 1
      if choice == 'p':
        if page > 1:
          return page - 1
        else:
          return page
      elif choice[0] == "p":
        try:
          page = int(choice[1:])
          if page > 0:
            return page
          else:
            return -1
        except Exception as e:
          print(e)
          return -1      
      if ',' in choice:
        choices = choice.split(',')
      else:
        choices = [choice]
      for choice in choices:
        select = results[int(choice) - 1]
        print(f"\n Downloading file for and adding {select.title} :\n")
        url = select.href
        download = requests.get(url)
        soup = BeautifulSoup(download.text, "html.parser")
        hash = soup.find(class_="infohash-box").span.text.strip()                                
        download_link = f"https://itorrents.org/torrent/{hash}.torrent"
        file = "/content/{}.torrent".format(select.title)
        !wget "$download_link" -O "$file" &> /dev/null
        i = 0
        if  i > 0 and attr.save_name:
          attr.save_name = attr.save_name + f" {str(i)}"
        attr.site_source = "1337x"
        add_local_files(attr)
        i += 1
      print("Done.\n")
      return 0
    else:
      print(f" An error occured while trying to access 1337x.com  status: {response.status_code}")
  return -1

if LT and not 'lt' in globals():
  !python -m pip install lbry-libtorrent &> /dev/null
  !apt install python3-libtorrent &> /dev/null
  !pip install rarbgapi &> /dev/null
  !pip3 install requests &> /dev/null
  !pip3 install beautifulsoup4 &> /dev/null

  import libtorrent as lt
  ses = lt.session()
  ses.listen_on(6881, 6891)
  User_Agent = '"Utorrent"/3.5.5' 
  sett = {
      'allow_multiple_connections_per_ip': True, 'active_downloads': Active_Downloads, 'active_seeds':1, 'active_checking': 3, 'active_limit':10, 'announce_to_all_tiers': True, 'announce_to_all_trackers':True, 'allow_multiple_connections_per_ip':True, 'auto_sequential':True, 'auto_manage_interval': 30, 'connections_limit':2330, 'connection_speed': 80, 'connections_slack':50, 'close_redundant_connections':True, 'file_pool_size':20, 'local_service_announce_interval':200, 'max_rejects':8, 'max_peerlist_size': 5000, 'max_paused_peerlist_size': 2000, 'max_failcount':3, 'min_announce_interval':30, 'min_reconnect_time': 20, 'peer_timeout':20, 'request_timeout':10, 'half_open_limit': 100, 'send_buffer_watermark':5*1024*1024, 'strict_end_game_mode':True, 'seed_time_limit': 60, 'tracker_completion_timeout':30, 'upload_rate_limit':0, #'user_agent': User_Agent,'inactivity_timeout':20, 'ban_web_seeds':False,
      }
  ses.apply_settings(sett)
  print("Client session started.")

  downloads_dict = {}
  active=[]
  completed = []
  uncompleted= []

  import requests
  from bs4 import BeautifulSoup
  watched_movies_list = []
  watched_list_url = "https://www.imdb.com/search/title/?title_type=feature&lists=ls047519430&view=simple&sort=alpha,asc&count=250&start="
  page1 = watched_list_url + "0"
  page2= watched_list_url + "251"
  reponse1 = requests.get(page1, headers = {"Accept-Language": "en-US"})
  scrap_watched_list(reponse1.text)
  reponse2 = requests.get(page2, headers = {"Accept-Language": "en-US"})
  scrap_watched_list(reponse2.text)

  import rarbgapi
  rarbg_movie_h264 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H264_1080P  
  rarbg_movie_h265 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H265_1080P
  rarbg_movie= [rarbg_movie_h264, rarbg_movie_h265]
  rarbg_tv= [rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES, rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES_HD]
  rarbg_all = rarbg_movie + rarbg_tv 

  def auto_reconnect(): 
    display(IPython.display.Javascript('''function ClickConnect(){btn = document.querySelector("colab-connect-button") if (btn != null){console.log("Click colab-connect-button"); btn.click() } btn = document.getElementById('ok') if (btn != null){console.log("Click reconnect"); btn.click() }} setInterval(ClickConnect,60000)'''))
  auto_reconnect()

print("Local temporary saving path  " + temp_save)
print("Local done saving path  " + done_path)
print("Drive main saving path:  " + drive_path)
print("Drive movie saving path:  " + Movies_Path)
print("Drive show saving path:  " + Shows_Path)
if Bak1: print("Drive bak1 path:  " + Bak1)
if Bak2: print("Drive bak2 path:  " + Bak2)
if Bak3: print("Drive bak3 path:  " + Bak3)
for item in uncompleted_sources:
  print(f"Uncomplete source: {item}")

# name_s = name_movie(name)  # path = Movies_Path + "/" + name_s   # return os.path.exists(path)   # shows_dir = filter_dirs(next(os.walk(Shows_Path))[1])   # shows = [show_name_trans(x) for x in shows_dir]   # name_s = show_name_trans(name)   # if name_s in shows :   #   return True    # else:   #   return False


#### Search

In [ ]:

import requests
from bs4 import BeautifulSoup 
import rarbgapi
rarbg = rarbgapi.RarbgAPI()
Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Site = "1337x" #@param ["1337x", "rarbg"] {type:"string"}
Search = "" #@param {type:"string"}
Page = 1 #@param {type:"integer"}
Category = "Movies" #@param ["All","Movies", "TV", "Apps", "Games"]
Sort = "By Seeders"  #@param ["By Seeders", "By Time", "By Size"]
Order = "Descending" #@param ["Descending", "Ascending"]
Save_Name = "" #@param {type:"string"}
Replace = False #@param {type:"boolean"}
Backup = True  #@param {type:"boolean"}
#@markdown 

sort = l337x_seeders
if Sort == "By Time":
  if Order == "Descending":
    sort = l337x_time_desc
  else:
    sort = l337x_time_asc
elif Sort == "By Size":
  if Order == "Descending":
    sort = l337x_size_desc
  else:
    sort = l337x_size_asc

category = "" if Category == "Any" else Category

attr = attrs()
attr.type_ = Type
attr.save_name = Save_Name
attr.replace = Replace
attr.backup = Backup

if Search:
  search(Site = Site, Search = Search, category = category, attr = attr, sort1337x = sort, Page = Page, auto = False)
   

#### IMDB

In [ ]:

#@markdown

from logging import exception
from re import I
import requests
from bs4 import BeautifulSoup 
import rarbgapi
rarbg = rarbgapi.RarbgAPI()
Type = Types.Movie #@param ["Types.Movie", "Types.Show", "None"] {type:"raw"}
Key_Word = "" #@param {type:"string"}
Years = "1960 1970" #@param {type:"string"}
Ratings=  "7.2-" #@param {type:"string"}
Min_Votes = "70000" #@param {type:"string"}
Max_Runtime_M = "200" #@param {type:"string"}
IMDB_Lists = "" #@param {type:"string"}
Exclude_Lists = "ls561569898" #@param {type:"string"}
IMDB_Sort = "By Date" #@param ["Alphabetically", "By Date", "By User Rating"]
IMDB_Order = "Ascending" #@param ["Ascending", "Descending"]
Top_250 = False #@param {type:"boolean"}
Top_1000 = True #@param {type:"boolean"}
Oscar_Winner = False #@param {type:"boolean"}
Best_Picture_Oscar = False #@param {type:"boolean"}
Best_Director_Oscar = False #@param {type:"boolean"}
English_Only = True #@param {type:"boolean"}
Animations = False #@param {type:"boolean"}
International_Movies = False #@param {type:"boolean"}
Indian_Movies = False #@param {type:"boolean"}
l337x_Download = True #@param {type:"boolean"}
l337x_Sort = l337x_seeders #@param ["l337x_seeders", "l337x_time_desc", "l337x_size_desc", "l337x_time_asc", "l337x_size_asc"]
Rarbg_Download = True #@param {type:"boolean"}
Rarbg_Auto = True  #@param {type:"boolean"}
Site_Category = "Movies" #@param ["Movies", "TV"]
Parameters = "1080p" #@param {type:"string"}
Filter_Downloaded = True #@param {type:"boolean"}
Filter_Watched = False #@param {type:"boolean"}
Replace = False #@param {type:"boolean"}
Backup = True  #@param {type:"boolean"}

desc_op = "Descending"
asc_op = "Ascending"
imdb_rating_op = "By User Rating"
imdb_alpha_op = "Alphabetically"
imdb_date_op = "By Date"
exclude="%21"
international = "ls560608427"
indian= "ls560456074"
animation = "ls560333050"
watched_movies ="ls047519430"
watched_series = "ls047701980"
movie_skip = ""
imdb = "https://www.imdb.com/title/"
imdb_date_sort = "release_date"
imdb_rating_sort = "user_rating"
imdb_alpha_sort = "alpha"
top250="top_250"
top1000= "top_1000"
oscar_winner="oscar_winner"
best_picture_oscar="best_picture_winner"
best_director_oscar="best_director_winner"
lists = imdb_labels = start_year = end_year = ""
ratings = Ratings.replace("-",",")

if '-' in Years:
  vals = Years.split("-")
  start_year = vals[0]
  end_year= vals[1]
elif ' ' in Years:
  vals = Years.split(" ")
  start_year = vals[0]
  end_year= vals[1]
else:
  start_year = Years
  end_year= Years

if Top_250:
  imdb_labels = imdb_labels + top250 + ","
if Top_1000:
  imdb_labels = imdb_labels + top1000 + ","
if Oscar_Winner:
  imdb_labels = imdb_labels + oscar_winner + ","
if Best_Picture_Oscar:
  imdb_labels = imdb_labels + best_picture_oscar + ","
if Best_Director_Oscar: 
  imdb_labels = imdb_labels + best_director_oscar 

if IMDB_Lists: lists = lists + IMDB_Lists + ","

if "," in Exclude_Lists: 
  for item in Exclude_Lists.split(","): 
    lists = lists + exclude + item + ","

elif Exclude_Lists: 
  lists = lists + exclude + Exclude_Lists + ","

if English_Only: 
  lists = lists + exclude + international + "," + exclude + indian + "," + exclude + animation + "&languages=en"
else:
  if International_Movies: 
    lists = international + ","
  if Indian_Movies: 
    lists = lists + indian 
  if Animations: 
    lists = lists + animation 

if IMDB_Sort == imdb_date_op: 
  imdb_sort = imdb_date_sort
elif IMDB_Sort == imdb_rating_op: 
  imdb_sort = imdb_rating_sort
elif IMDB_Sort == imdb_alpha_op: 
  imdb_sort = imdb_alpha_sort
else: imdb_sort = imdb_date_sort

if IMDB_Order == asc_op : 
  imdb_sort = imdb_sort + ",asc"
else: 
  imdb_sort = imdb_sort + ",desc" 

if Type == Types.Movie:
  title_type = "feature"
elif Type == Types.Show:
  title_type = "tv_series"
else:
  title_type = ""

attr0 = attrs()
attr0.type_ = Type
attr0.replace = Replace
attr0.backup = Backup

params = imdb_params(keyword = Key_Word, title_type = title_type, s_year = start_year, e_year = end_year, ratings = ratings, min_votes = Min_Votes, max_run = Max_Runtime_M, lists = lists, labels = imdb_labels, imdb_sort =  imdb_sort, filter_watched = Filter_Watched, filter_downloaded = Filter_Downloaded, l337x_download = l337x_Download, rarbg_download = Rarbg_Download , site_category = Site_Category, l337x_sort = l337x_Sort, download_params = Parameters, rarbg_auto = Rarbg_Auto)
AdvancedIMDBSearch(params, attr0)


#### Start

In [ ]:

#@markdown

import time
from IPython.display import display
import ipywidgets as widgets
downloads= ses.get_torrents()
ses.resume()
ses.start_dht()
completed_views = []
active_views = []

title_labels = []
manage_checks = []
progress_labels = []
progress_bars = []
states = []
type_labels = []
download_speeds = []
upload_speeds = []
total_downloads = []
total_bytes = []
total_uploads = []

state_str = ["Queued","Checking","Downloading metadata","Downloading","Finished","Seeding","Allocating","Checking resume files"]

style0= {"padding-left":"200px","padding-right":"200px"}
layout_15 =widgets.Layout(width='auto', flex='15 1 0%', align_items="center")
layout_10 =widgets.Layout(width='auto', flex='10 1 0%', align_items="center")
layout_8 =widgets.Layout(width='auto', flex='8 1 0%', align_items="center")
layout_7 =widgets.Layout(width='auto', flex='7 1 0%', align_items="center")
layout_5 =widgets.Layout(width='auto', flex='5 1 0%', align_items="center")
layout_4 =widgets.Layout(width='auto', flex='4 1 0%', align_items="center")
layout_3 =widgets.Layout(width='auto', flex='3 1 0%', align_items="center")
layout_2 =widgets.Layout(width='auto', flex='2 1 0%', align_items="center")
layout_1 =widgets.Layout(width='auto', flex='1 1 0%', align_items="center")
layout_1E =widgets.Layout(width='auto', flex='1 1 0%', align_items="flex-start")

layout_up = {'flex_flow':'row','align_items':'center','width':'100%','justify_content':'space-around'}
layout_manage = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','height':'50px','justify_content':'flex-start'}
layout_r = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','justify_content':'space-around'}
layout_c = {'width':'95%','align_items':'center','border':'solid #d9dbda 1px'}
layout_v = {'flex_flow':'row','align_items':'center','width':'95%','justify_content':'space-around'}

empty_label_1 = widgets.Label(value = "", layout = layout_1)
empty_label_2 = widgets.Label(value = "", layout = layout_2)
empty_label_3 = widgets.Label(value = "", layout = layout_3)
empty_label_4 = widgets.Label(value = "", layout = layout_4)
empty_label_5 = widgets.Label(value = "", layout = layout_5)

total_torrents_label = widgets.Label(value="Total torrents:  {0}".format(len(downloads)),layout=layout_3)
active_label = widgets.Label(value="Active:  0",layout=layout_3)
dead_label = widgets.Label(value="Dead:  0", layout=layout_3)
completed_label = widgets.Label(value="Completed:  {0}".format(len(completed)),layout=layout_3)

total_download_label=widgets.Label(value = "Session download:  {0}".format(data_str(0)),layout=layout_5)
total_progress_label=widgets.Label(value = "{0}%".format(0),layout=layout_3)
total_upload_label=widgets.Label(value = "Session upload:  {0}".format(data_str(0)),layout=layout_5)

def Download_View(handle, type_, hide = False):
  status = handle.status()

  progress_bar =widgets.FloatProgress(
      value= status.progress*100,
      min=0,
      max=100,
      step=0.5,
      bar_style='info',
      orientation='horizontal',
      layout=layout_15)
  title_label = widgets.Label(value=handle.name(), layout = layout_10)
  progress_label= widgets.Label(value="{:.2f}%".format(status.progress*100), layout=layout_1)
  title_labels.append(title_label)
  progress_bars.append(progress_bar)
  progress_labels.append(progress_label)

  strs=[
        "Download Speed:    {0}".format(speed_str(status.download_rate)),
        "Upload speed:    {0}".format(speed_str(status.upload_rate)),
        "Total Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Total Upload:    {0}".format(data_str(status.total_upload)),       #All time upload dowsn't work properly
        "      {0} / {1}".format(status.total_wanted_done, status.total_wanted)
  ]

  ls = [widgets.Label(value = v, layout = layout_4) for v in strs]
  ls.insert(0,widgets.Label(value = state_str[status.state], layout=layout_2))
  ls.insert(1,widgets.Label(value = type_.name, layout=layout_2))

  states.append(ls[0])
  type_labels.append(1)
  download_speeds.append(ls[2])
  upload_speeds.append(ls[3])
  total_downloads.append(ls[4])
  total_uploads.append(ls[5])
  total_bytes.append(ls[6])  

  ls.append(empty_label_1)

  r1 = widgets.Box([ title_label, progress_label, progress_bar, empty_label_1], layout = layout_r, style = style0)
  r2 = widgets.Box(ls, layout = layout_r, style = style0)
  c = widgets.VBox([r1,r2], layout= layout_c)
  if hide:
    c.layout.display = 'none'
  return c

def Complete_View(handle, type_):
  status = handle.status()
  progress_bar = widgets.FloatProgress(value = 100, min = 0, max = 100, step = 50, bar_style = 'success', orientation = 'horizontal', layout = layout_15 )
  download_label = widgets.Label(value = handle.name(), layout = layout_10)
  progress_label= widgets.Label(value = "100%", layout = layout_1E)
  upload = status.all_time_upload
  if upload == 0:
    upload = status.total_upload
  strs=[
        "Completed at:    {0}".format(time.strftime('%I:%M:%S %p', time.localtime(status.completed_time))),
        "Total time:     {:.2f} m".format(status.active_time/60),
        "Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Upload:    {0}".format(data_str(upload)) ]

  ls = [widgets.Label(value = v, layout = layout_4) for v in strs]
  ls.insert(0, widgets.Label(value = "Completed", layout = layout_2))
  ls.insert(1, widgets.Label(value = type_.name, layout = layout_2))
  ls.insert(6, empty_label_3)
  r1 = widgets.Box([ download_label,progress_label, progress_bar,empty_label_1],layout= layout_r,style= style0)
  r2 = widgets.Box(ls,layout= layout_r,style=style0)
  c = widgets.VBox([r1,r2],layout= layout_c)
  return c

def Update_View(i, status, type_, show = False):
  if show:
    active_views[i].layout.display = 'flex'
  title_labels[i].value = status.name
  states[i].value = state_str[status.state]
  progress_labels[i].value =  "{:.2f}%".format(status.progress*100)
  progress_bars[i].value = status.progress*100
  type_labels[i] = type_.name
  download_speeds[i].value = "Download Speed:    {0}".format(speed_str(status.download_rate))
  upload_speeds[i].value = "Upload speed:    {0}".format(speed_str(status.upload_rate))
  total_downloads[i].value = "Download:    {0} / {1}".format(data_str(status.total_wanted_done), data_str(status.total_wanted))
  total_bytes[i].value = "      {0} / {1}".format(status.total_wanted_done, status.total_wanted)
  total_uploads[i].value = "Upload:    {0}".format(data_str(status.total_upload))
  if show:
    active_views[i].layout.display = 'flex'
  return

total_download = total_wanted = total_up = completed_down = completed_up = total_progress = 0
def Update_Stats(_download,_upload):
  if total_wanted == 0:
    total_progress = 0
  else:
    total_progress = (100*_download)/total_wanted 

  active_label.value = "Active:  {}".format(len(active) - dead)
  completed_label.value = "Completed:  {}".format(len(completed))
  dead_label.value = "Dead:  {}".format(dead)
  total_download_label.value = f"Session download:   {data_str(_download)} / {data_str(total_wanted)}"
  total_progress_label.value = f"     {round(total_progress,2)}%"
  total_upload_label.value = "Session upload:   {}".format(data_str(_upload))

def Remove_Active_View(i):
  active_views[i].layout.display = 'none'
  del active_views[i]
  del title_labels[i]
  del progress_labels[i]
  del progress_bars[i]
  del type_labels[i]
  del download_speeds[i]
  del upload_speeds[i]
  del total_downloads[i]
  del total_bytes[i]
  del total_uploads[i]
  return

def Completed_Stats():
  _download = 0
  _upload = 0
  for c in completed:
    status = c.status()
    upload = status.all_time_upload
    if upload == 0:
      upload = status.total_upload  
    _download += status.total_wanted_done
    _upload += upload
  return [_download,_upload]

dead = 0
active.sort(key = lambda obj: obj.status().progress, reverse = True)
for handle in active:
  status = handle.status()
  total_wanted += status.total_wanted
  if status.total_wanted_done == 0:
    hide = True
    dead += 1
  else:
    hide = False
  view = Download_View(handle, handle.attr.type_, hide)
  active_views.append(view)
  handle.progress = status.progress
  handle.resume()
  
for handle in completed:
  key =  str(handle.info_hash())
  type_ = downloads_dict[key].attr.type_
  completed_views.append(Complete_View(handle, type_))
    
active_down = active_up = 0 
stat = Completed_Stats()
completed_down = stat[0]
completed_up = stat[1]

Update_Stats(completed_down, completed_up)
bakstr = ""
if Bak1: bakstr += "Bak1: " + Bak1
if Bak2: bakstr += "         Bak2: " + Bak2
if Bak3: bakstr += "         Bak3: " + Bak3
print("\n\nSESSION IS RUNNING             General save: {}          Temp save: {}          {}".format(drive_path, temp_save, bakstr))
stats_row = widgets.Box([total_torrents_label, active_label, completed_label, dead_label, total_download_label, total_progress_label, total_upload_label, empty_label_3], layout=layout_up, style=style0)
status_log =  widgets.Label(value = ".", layout=layout_up, style=style0 )
display(stats_row, status_log, *active_views)
print("\n")
display(*completed_views)

try:
  while active:
    active_down = active_up = active_wanted = 0
    for index in range(len(active)):
      handle = active[index]    
      status = handle.status()
      donebytes = status.total_wanted_done
      wantedbytes = status.total_wanted
      handle.progress = status.progress
      if donebytes > 70000 and active_views[index].layout.display == 'none':
        show = True
        dead -= 1
      else:
        show = False
      Update_View(index, status, handle.attr.type_, show)
      if handle.is_finished() or (wantedbytes > 10000 and donebytes == wantedbytes):
        handle.pause()
        attr: attrs = handle.attr
        name = handle.name()
        newname = attr.save_name      
        p1 = status.save_path + "/" + name
        p0 = status.save_path
        if not os.path.isdir(p1):
          p0 = status.save_path + "/" + os.path.splitext(name)[0]
          os.makedirs(p0, exist_ok = True)
          !mv "$p1" "$p0"
          p1 = p0
        shave(handle, p0, p1)
        clean_download(attr.type_, p1, newname)
        p2 = done_path + "/" + newname
        !mv "$p1" "$p2"

        if not newname:
          newname = attr.name
          p2 = p2 + name

        if attr.type_ == Types.Episode and os.path.isdir(p2):
          p2 = p2 + "/" 

        status_log.value = f"\nSaving {newname} to: {attr.save_path}"
        !rsync -I -r "$p2" "$attr.save_path"

        if attr.backup:
          if attr.baksv1 :
            status_log.value = f"Copying {newname} to: {attr.baksv1}"
            !rsync -I -r "$p2" "$attr.baksv1"
          if attr.baksv2:
            status_log.value = f"Copying  {newname} to: {attr.baksv2}"
            !rsync -I -r "$p2" "$attr.baksv2"
          if attr.baksv3:
            status_log.value = f"Copying  {newname} to: {attr.baksv3}"
            !rsync --remove-source-files -I -r "$p2" "$attr.baksv3"

        if attr.uncomplete_source and os.path.exists(attr.uncomplete_source):
          !rm -r "$attr.uncomplete_source"
          
        Remove_Active_View(index)
        status_log.value = "."
        completed.append(handle)
        active.remove(handle)
        handle.completed = True
        complete_view = Complete_View(handle, attr.type_)
        completd_stat = Completed_Stats()
        completed_down = completd_stat[0]
        completed_up = completd_stat[1]
        display(complete_view)
        break
      else:
        active_up += status.total_upload
        active_down += status.total_wanted_done
        active_wanted += status.total_wanted
    total_wanted = completed_down + active_wanted
    total_download = completed_down + active_down
    total_up = active_up + completed_up
    Update_Stats(total_download, total_up)
    active.sort(key = lambda obj: obj.progress, reverse= True)
    time.sleep(0.9)
except Exception as e:
  print(e)
except KeyboardInterrupt as k:
  print(k)


#### Pause

In [ ]:

#@markdown 

downloads= ses.get_torrents()
for h in downloads:
  try:
    print(h.name())
    h.pause()
  except:
    print("Invailed torrent handle.")
    
ses.pause()

#### Resume

In [ ]:

#@markdown 

ses.resume()
for h in active:
  h.resume()

#### Save

In [ ]:

#@markdown 

Save_To = "/content/drive/Shareddrives/SM1/Downloads/.uncomplete" #@param {type: "string"}
def sv():
  if active:
    print(" Choose a download to save progress , q to exit:\n")
    for i in range(len(active)):
      print("   {0}-  {1}".format(i+1, active[i].name()))
    print()
    inpt =  input().lower()
    if inpt == '' or inpt == " " or inpt == 'q' or inpt == "s":
      return
    if "," in inpt:
       choices = inpt .split(",")
    else:
      choices = [inpt]

    for choice in choices:
      choice = int(choice) - 1
      if(choice >= len(active) or choice < 0):
        print("\nThe choice is out of range.\n")
      else:
        name = active[choice].name()
        print("\nSaving {0} into {1}\n".format(name, Save_To))
        os.makedirs(Save_To, exist_ok = True)
        path = temp_save +"/"+ name
        !cp -r "$path" "$Save_To"
        print("..................")
        print("       Done")
sv()

#### Files

In [ ]:

#@markdown

handle = active[-1]

Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  for index in range(len(files)):
    if index >= Start and index <= End:
      handle.file_priority(index,5)
    else:
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

      
  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")


In [ ]:

#@markdown

handle = active[-1]

Second_Range=False #@param {type:"boolean"}
Range1_Start=0 #@param {type:"integer"}
Range1_End=0 #@param {type:"integer"}

#@markdown 

Third_Range=False #@param {type:"boolean"}
Range2_Start=0 #@param {type:"integer"}
Range2_End=0 #@param {type:"integer"}

#@markdown 

Exlude_Range=False #@param {type:"boolean"}
Exlude_Range_Start=0 #@param {type:"integer"}
Exlude_Range_End=0 #@param {type:"integer"}


if(handle.has_metadata()):
  file_count = len(handle.file_priorities())
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for index in range(file_count):
    if Second_Range and (index >= Range1_Start and index <= Range1_End):
      handle.file_priority(index,5)

    if Third_Range and (index >= Range2_Start and index <= Range2_End):
      handle.file_priority(index,5)

    if Exlude_Range and (index >= Exlude_Range_Start and index <= Exlude_Range_End):
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")

In [ ]:

#@markdown 

s_list=["S01","S02","S03","S04","S05","S06","S07","S08","S09","S10","S11","S12"]
check_list =["False","False","False","False","False","False","False","False","False","False","False","False"]
ok_list = [ ]
handle = active[-1]

S01=False #@param {type:"boolean"}
S02=False #@param {type:"boolean"}
S03=False #@param {type:"boolean"}
S04=False #@param {type:"boolean"}
S05=False #@param {type:"boolean"}
S06=False #@param {type:"boolean"}
S07=False #@param {type:"boolean"}
S08=False #@param {type:"boolean"}
S09=False #@param {type:"boolean"}
S10=False #@param {type:"boolean"}
S11=False #@param {type:"boolean"}
S12=False #@param {type:"boolean"}

check_list[0]=S01
check_list[1]=S02
check_list[2]=S03
check_list[3]=S04
check_list[4]=S05
check_list[5]=S06
check_list[6]=S07
check_list[7]=S08
check_list[8]=S09
check_list[9]=S10
check_list[10]=S11
check_list[11]=S12

for index, season in enumerate(s_list[:]):
  if check_list[index]:
    ok_list.append(season)

if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for x in range(files.num_files()):
    if any(i in files.file_name(x) for i in ok_list):
      handle.file_priority(x,5)
    else:
      handle.file_priority(x,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))

  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("    Metadata not recieved yet.")


#### Clear Completed

In [ ]:

#@markdown 

for download in completed:
  try:
    print("Deleting: " + download.name())
    ses.remove_torrent(download)
  except:
    print("Invild torrent.")
completed.clear()

#### Clear Active

In [ ]:

#@markdown 

for download in active:
  download.pause()
  active.remove(download)
  try:
    ses.remove_torrent(download)
  except:
    print("Invalid torrent.")
  
active.clear()

#### Clear All

In [ ]:

#@markdown 

downloads= ses.get_torrents()

for download in downloads:
    print("Deleting: " + download.name())
    ses.remove_torrent(download, 0)

active.clear()
completed.clear()
downloads_dict.clear()

#### Files Manager

In [ ]:

#@markdown 

From_Path = "/content/done/Modern Family 2009/S05" #@param ["", "/content/drive/Shareddrives/SM1/Library/Movies", "/content/drive/Shareddrives/SM1/Library/Music", "/content/drive/Shareddrives/SM1/Library/TV Shows", "/content/drive/Shareddrives/SM2/Library/Movies", "/content/drive/Shareddrives/SM2/Library/Music", "/content/drive/Shareddrives/SM2/Library/TV Shows","/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}
To_Path = "/content/drive/Shareddrives/SM1/Library/TV Shows/Modern Family 2009/S05"  #@param ["", "/content/drive/Shareddrives/SM2/Library/Movies", "/content/drive/Shareddrives/SM2/Library/Music", "/content/drive/Shareddrives/SM2/Library/TV Shows", "/content/drive/Shareddrives/SM1/Library/Movies", "/content/drive/Shareddrives/SM1/Library/Music", "/content/drive/Shareddrives/SM1/Library/TV Shows","/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}
Then_Copy_To = "/content/drive/Shareddrives/SM2/Library/TV Shows/Modern Family 2009/S05"  #@param ["", "/content/drive/Shareddrives/SM3/Library/Movies", "/content/drive/Shareddrives/SM3/Library/Music", "/content/drive/Shareddrives/SM3/Library/TV Shows", "/content/drive/Shareddrives/SM2/Library/Movies", "/content/drive/Shareddrives/SM2/Library/Music", "/content/drive/Shareddrives/SM2/Library/TV Shows","/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}
And_Then_To = "/content/drive/Shareddrives/SM3/Library/TV Shows/Modern Family 2009/S05"  #@param ["", "/content/drive/Shareddrives/SM4/Library/Movies", "/content/drive/Shareddrives/SM4/Library/Music", "/content/drive/Shareddrives/SM4/Library/TV Shows", "/content/drive/Shareddrives/SM3/Library/Movies", "/content/drive/Shareddrives/SM3/Library/Music", "/content/drive/Shareddrives/SM3/Library/TV Shows","/content/drive/MyDrive/Library/Movies", "/content/drive/MyDrive/Library/TV Shows", "/content/drive/MyDrive/Library/Music"] {allow-input: true}
#@markdown

Delete = False #@param {type:"boolean"}
Process_Backup = True #@param {type:"boolean"}
To_Replace = True #@param {type:"boolean"}
#@markdown

Move = False #@param {type:"boolean"}
Only_Rename = False #@param {type:"boolean"}
Movie_Name = False #@param {type:"boolean"}
Show_Name = False #@param {type:"boolean"}

def cp_mv(content, from_path, to_path, then_copy_to = "", and_then_to = "", move = False, only_rename = False, movie_name = False, show_name = False):
  os.makedirs(to_path, exist_ok= True)
  if then_copy_to :
    os.makedirs(then_copy_to, exist_ok= True)
  if and_then_to :
    os.makedirs(and_then_to, exist_ok= True)

  if content:
    print("\n Enter directory range to copy [Format: start-end  All: . Exit: q]:\n")
    print_files(content, 0 ,len(content))
    print()
    start = end = -1
    select = input().lower()
    if select == '' or select == ' ' or select == 'q' or select == 's':
       return
    if move: print(f"\nMoving files to: {to_path}\n")
    elif only_rename: print(f"\nRenaming files:\n")
    else: print(f"\nCopying files to {to_path}\n")

    if select == '.':
      rngs = [f"1-{len(content)}"]
    else:
      rngs = select.split(',')
    for rng in rngs:
      if '-' in rng:
        rngs = rng.split('-')
        start = int(rngs[0])-1
        end = int((rngs[1]))
      else:
        end = int(rng)
        start = end - 1

      print("Range: {0}-{1}\n".format(start+1,end))
      if end > len(content) or start < 0 or ((start + end) < 0) :
        print("The ranges' ends are out of the directory file range.\n")
        continue
      elif only_rename:
        print_files(content,start,end)
        print()
        for i in range(start, end):
          name  = ""
          if movie_name:
            name = name_movie(content[i])
          elif show_name:
            name = name_show(content[i])
          else:
            print(" Enter a new name:\n")
            name = input()
          if not name:
            continue
          print(" Renaming   {0}   new name:   {1}\n".format(content[i], name))
          p1 = from_path + "/" + content[i]
          p2 = from_path + "/" + name 
          !mv "$p1" "$p2"
        
      elif Move:
        print_files(content,start,end)
        print()
        for i in range(start, end):
          rename  = ""
          if Movie_Name:
            rename = "/" + name_movie(content[i])
          elif Show_Name:
            rename =  "/" + name_show(content[i])
          print("   Moving  {0}".format(content[i]))
          p1 = from_path + "/" + content[i]
          p2 = to_path +  rename
          !mv "$p1" "$p2"
          
          if not rename:
            p2 = p2 + "/" + content[i]
          if then_copy_to:
            !cp -r "$p2" "$then_copy_to"
          if and_then_to:
            !cp -r "$p2" "$and_then_to"

      else:
        print_files(content,start,end)
        print()
        for i in range(start, end):
          rename  = ""
          if Movie_Name:
            rename = "/" + name_movie(content[i])
          elif Show_Name:
            rename =  "/" + name_show(content[i])    

          p0 = to_path + content[0]
          p1 = from_path + "/" + content[i]
          p2 = to_path + rename

          if os.path.exists(p0) or (rename and os.path.exists(p2)) :
            print("  Skipping:  {0}  directory alraedy exists.".format(content[i]))
          else:
            print("  Copying:  {} to {}".format(content[i], p2))

            !rsync  -P -h -r --size-only "$p1" "$p2"
          if then_copy_to :
            p0 = then_copy_to + content[0]
            p1 = from_path + "/" + content[i]
            p2 = then_copy_to + rename
            if not (os.path.exists(p0) or (rename and os.path.exists(p2))) : 
              print("  Copying:  {} to {}".format(content[i], p2)) 
              !cp -r "$p1" "$then_copy_to"
          if and_then_to:
            p0 = and_then_to + content[0]
            p1 = from_path + "/" + content[i]
            p2 = and_then_to + rename
            if not (os.path.exists(p0) or (rename and os.path.exists(p2))) :
              print("  Copying:  {} to {}".format(content[i], p2)) 
              !cp -r "$p1" "$and_then_to"          

    print("\n   .....................\n           Done\n")
    return
  else:
    print("Empty directory.\n")
    return

def rm(content, dp, baktoo, rp):
  if content:
    print(" Enter directory range to copy (start-end):")
    print_files(content, 0 ,len(content))
    print()
    start = end = 0
    rng = input()
    print()
    if rng == 'q':
      return
    if not '-' in rng:
      start = int(rng) - 1
      end = start + 1
    elif '.' == rng:
      start = 0
      end = len(rng)
    else:
      rngs = rng.split('-')
      start = int(rngs[0]) - 1
      end = int((rngs[1]))
    print("Chosen range start: {0}, end: {1}".format(start+1,end))
    print()
    if end > len(content) or start < 0 :
      print("The ranges' ends are out of the directory file range.")
      return
    elif rp:
      for i in range(start, end):
        print("Moving to .replace:  {0}".format(content[i]))
        path = dp +"/"+content[i] 
        replace_download(path)
        if baktoo:
          pt = getbak(path, Bak1)    
          print(f"Processing: {pt}")
          replace_download(pt)
          pt = getbak(path, Bak2)    
          print(f"Processing: {pt}")
          replace_download(pt)
          pt = getbak(path, Bak3)    
          print(f"Processing: {pt}")
          replace_download(pt)
    else:
      print("Deleting these directories".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        print("   Deleting  {0}".format(content[i]))
        path = dp +"/"+content[i]
        !rm -r "$path"
        if baktoo:
          pt = getbak(path, Bak1)    
          print(f"Processing: {pt}")
          if pt: 
            !rm -r "$pt"
          
          pt = getbak(path, Bak2)    
          print(f"Processing: {pt}")
          if pt: 
            !rm -r "$pt"
          
          pt = getbak(path, Bak3)    
          print(f"Processing: {pt}")
          if pt: 
            !rm -r "$pt"

    print("......................")
    print("        Done")
    return
  else:
    print("Empty directory.")
    return

wlk1 = filter_dirs(next(os.walk(From_Path))[1])
wlk2 = next(os.walk(From_Path))[2]
wlk1.sort()
wlk2.sort()
pts = wlk1 + wlk2

if Delete:
  rm(pts, From_Path, Process_Backup, To_Replace)
else:
  cp_mv(pts, From_Path, To_Path, Then_Copy_To, And_Then_To, Only_Rename, Movie_Name, Show_Name)


#### Temp

In [ ]:

#@markdown
  
Delete_torrent_files = True #@param {type:"boolean"}
Clear_done_folder = True #@param {type:"boolean"}
Clear_temp_folder = False #@param {type:"boolean"}

if Delete_torrent_files:
  !find . -maxdepth 1 -name "*.torrent" -type f -delete 

if Clear_done_folder:  
  !rm -r "/content/done/"*

if Clear_temp_folder:
  !rm -r "$temp_save/"*  

#### Remove

In [ ]:

#@markdown

downloads= ses.get_torrents()

print("Please select a torrent to remove:")
print("")

for index,h in enumerate(downloads[:]):
  print("{0}- {1}".format(index + 1, h.name()))

print("Enter torrent selection,  q to exit.")
select = input("").lower()
if select != "q" or select != "s":
  if "," in select:
    selects = select.split(",")
  else:
    selects = [select]

  for select in selects:
    select = int(select) - 1
    h = downloads[select]
    print("\nDeleting {0}".format(h.name()))
    print("...........\n")
    h.pause()
    if h in active:
      active.remove(h)
    elif h in completed:
      active.remove(h)
    try:
      ses.remove_torrent(h)
      del downloads_dict[str(h.info_hash())]
    except:
      print("Selected torrent is invalid.")  
  print("Done.")

#### Clean

In [ ]:

Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}

def clean_general(Type) :
    dirs = next(os.walk(drive_path))[1]
    if checkpoint in dirs: dirs.remove(checkpoint)
    for name in dirs:
      newname = ""
      
      p1 = drive_path + "/" + name
      p0 = drive_path
      if not os.path.isdir(p1):
        p0 = drive_path + "/" + os.path.splitext(name)[0]
        os.makedirs(p0, exist_ok= True)
        !mv "$p1" "$p0"
        p1 = p0

      if Type == Types.Movie:
          newname = name_movie(name)
          save_path = Movies_Path
          after_shave(p1, newname)
      elif Type == Types.Show:
          newname = name_show(name)
          clean_tv(p1)
          save_path = Shows_Path
          check = save_path + "/" + newname
      elif Type == Types.Season:
          newname = name_season(name)
          if not newname:
            newname = name
          show = name_show(name)
          clean_season(p1)
          save_path= Shows_Path + "/" + show
          os.makedirs(save_path, exist_ok= True)
      elif Type == Types.Episode:
          newname = name_episode(name)
          show = name_show(name)
          season= name_season(name)
          if not season:
            season = name
          save_path= Shows_Path + "/" + show + "/" + season
          os.makedirs(save_path, exist_ok= True)
      else:
          save_path = drive_path

      if newname:
        p2 = drive_path + "/" + newname
        !mv "$p1" "$p2"
      else:
        p2 = p2 + name

      if Type == Types.Episode and os.path.isdir(p2):
        clean_season(p2)
        p02 = p2
        p2 = p2 + "/" 
        
      if Type == Types.Show and os.path.isdir(check):
        save_path = check
        p02 = p2
        p2 = p2 + "/"

      !mv "$p2"* "$save_path"
      if p02 and os.path.exists(p02): 
        !rm -r "$p02"
      
clean_general(Type)

In [ ]:

#@markdown

Path = "" #@param {type: "string"}
Movie_Dir =  "" #@param {type: "string"}
if Path and os.path.exists(Path):
  name = Path.split("/")[-1]
  print(f"Cleaning movie: {Path}")
  clean_movie(Path, name)
  if Bak1:
    path = getbak(Path, Bak1)
    if os.path.exists(path):
      print(f"Cleaning movie: {path}")
      clean_movie(path, name)
    else: 
      print(f"Bak path: {path} doesn't exist.")

  if Bak2:
    path = getbak(Path, Bak2)
    if os.path.exists(path):
      print(f"Cleaning movie: {path}")
      clean_movie(path, name)
    else: 
      print(f"Bak path: {path} doesn't exist.")

  if Bak3:
    path = getbak(Path, Bak3)
    if os.path.exists(path):
      print(f"Cleaning movie: {path}")
      clean_movie(path, name)
    else: 
      print(f"Bak path: {path} doesn't exist.")
    print("Done.")
elif Movie_Dir:   
  clean_movies_dir(Movie_Dir)
else:
  clean_movies_dir()

In [ ]:

#@markdown

Path = "" #@param {type: "string"}
Keep_Char_In_Eps = "" #@param {type: "string"}
Clean_Backup = True #@param {type: "boolean"}
if Path and os.path.exists(Path):
  print(f"Cleaning TV path: {Path}")
  clean_tv(Path, Keep_Char_In_Eps)
  if Clean_Backup:
    showname = os.path.basename(os.path.normpath(Path))
    if Bak1:
      bakpt = baktv1 + "/" + showname
      print(f"Cleaning backup TV path: {bakpt}")
      clean_tv(bakpt, Keep_Char_In_Eps)
    if Bak2:
      bakpt = baktv2 + "/" + showname
      print(f"Cleaning backup TV path: {bakpt}")
      clean_tv(bakpt, Keep_Char_In_Eps)
    if Bak3:
      bakpt = baktv3 + "/" + showname
      print(f"Cleaning backup TV path: {bakpt}")
      clean_tv(bakpt, Keep_Char_In_Eps)
else:
  print(f"Cleaning TV parent path: {Shows_Path}")
  clean_shows()
  if Clean_Backup:
    if Bak1:
      print(f"Cleaning backup parent TV path: {baktv3}")
      clean_shows(baktv1)
    if Bak2:
      print(f"Cleaning backup parent TV path: {baktv3}")
      clean_shows(baktv2)
    if Bak3:
      print(f"Cleaning backup parent TV path: {baktv3}")
      clean_shows(baktv3)

#### Spotify

In [ ]:

#@markdown

import os, uuid, urllib.parse, re
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files
from datetime import datetime
import pytz
tz = pytz.timezone('Turkey')

SavePathYT = widgets.Dropdown(options=["/content/downloads", "/content"])
temp= "/content/downloads"
os.makedirs('tools/spotify-downloader/', exist_ok=True)
os.makedirs(temp, exist_ok= True)
dirs = glob("/content/drive/Shared drives/*")
for dir in dirs:
  if not os.path.exists(dir):
    os.makedirs(dir + "/Downloads")

# # Config files
# data = """spotify-downloader:
#   avconv: false
#   download-only-metadata: false
#   dry-run: false
#   file-format: '{artist} - {track_name}'
#   folder: /home/user/Music
#   input-ext: .m4a
#   log-level: INFO
#   manual: false
#   music-videos-only: false
#   no-fallback-metadata: false
#   no-metadata: false
#   no-spaces: false
#   output-ext: .mp3
#   overwrite: prompt
#   search-format: '{artist} - {track_name} lyrics'
#   skip: null
#   spotify_client_id: 4fe3fecfe5334023a1472516cc99d805
#   spotify_client_secret: 0f02b7c483c04257984695007a4a8d5c
#   trim-silence: false
#   write-successful: null
#   write-to: null
#   youtube-api-key: null
# """
# with open('tools/spotify-downloader/config.yml', 'w') as wnow:
#   wnow.write(data)

Links = widgets.Textarea(placeholder='''Link list
(one link per line)''')

fileFormat = widgets.Text(
    value='{artist} - {track_name}',
    placeholder='File name format',
    description="""File Name: file format to save the downloaded track with, each
                        tag is surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

searchFormat = widgets.Text(
    value='{artist} - {track_name} lyrics',
    placeholder='Search format',
    description="""Search Format: search format to search for on YouTube, each tag is
                        surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

playlistName = widgets.Text(
    value='',
    placeholder='Playlist Name',
    description="Playlist Name:",
    disabled=False
)

tab = widgets.Tab()

Extension = widgets.Select(options=["aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"], value="mp3")
writeM3u = widgets.ToggleButton(
    value=False,
    description='Write .m3u playlist',
    disabled=False,
    button_style='',
    tooltip='''generate an .m3u playlist file with youtube links
               given a text file containing tracks''',
    icon='check'
)

ExtraArg = widgets.Text(placeholder="Extra Arguments")
class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = [Music_Path, "/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/Downloads")
    else:
      SavePathYT.options = [Music_Path, "/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = [Music_Path, "/content/downloads", "/content"]
  SavePathYT.value = Music_Path

out = widgets.Output()
def ShowYT():
  clear_output(wait = True)
  mainTab = widgets.Box([widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                       searchFormat, fileFormat, playlistName, widgets.HBox([ writeM3u])]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])])])
  tab.children = [mainTab]
  tab.set_title(0, 'spotify-downloader')
  display(tab)
  display(HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"), SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))
  display(out)

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    if writeM3u.value:
      M3u = '--m3u'
    else:
      M3u = ''
    args=""
    for Link in Links.value.splitlines():
        args = args + f" {Link}"
    extraargC = ExtraArg.value
    dlname= playlistName.value
    if not dlname:
      now = datetime.now(tz).strftime("%H%M%S")
      dlname = f"dl{now}"
    tempsave = temp + "/" + dlname
    os.makedirs(tempsave, exist_ok= True)
    savepath = SavePathYT.value
    with out:
      print("Downloading..")
      cmd = f"spotdl {args} -o \"{tempsave}\" --of {Extension.value} {M3u} > spotdl_log.txt" 
      !$cmd 
      print("Saving to to {} ..".format(savepath))
      !mv "$tempsave" "$savepath"
      bp = savepath + "/" + dlname
      if Bak1 and os.path.exists(bp):
        print("Copying to {} ..".format(bakmc1))
        !cp -r "$bp" "$bakmc1"

      if Bak2 and os.path.exists(bp):
        print("Copying to {} ..".format(bakmc2))
        !cp -r "$bp" "$bakmc2"

      if Bak2 and os.path.exists(bp):
        print("Copying to {} ..".format(bakmc3))
        !cp -r "$bp" "$bakmc3"
    clear_output()
    ShowYT()

if not os.path.isfile("/usr/local/bin/spotdl"):
  #!add-apt-repository ppa:savoury1/ffmpeg4 --yes &> /dev/null
  !add-apt-repository ppa:savoury1/ffmpeg4 --yes &> /dev/null
  !apt-get update &> /dev/null
  !apt-get install ffmpeg &> /dev/null
  get_ipython().system_raw("pip3 install spotdl && apt-get install ffmpeg > out.txt")

RefreshPathYT()
ShowYT()

#### Youtube

In [ ]:

#@markdown 

Archive = False #@param {type:"boolean"}

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/Downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/Downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

#### Settings

In [ ]:

#@markdown

Announce_To_All_Tiers = True 
Announce_To_All_Trackers = True 
Active_Limit = 5 #@param {type:"integer"}
Active_Downloads = 3 #@param {type:"integer"}
Active_Seeds = 1 
Active_Checking = 3 #@param {type:"integer"}
Connections_Limit = 2330 
Connection_Speed = 80 
Connections_Slack = 50 
File_Pool_Size = 20 
Half_Open_Limit = 100 
Max_Rejects = 8 
Max_Peerlist_Size = 5000 
Max_Paused_Peerlist_Size = 2000 
Max_Failcount = 3 
Min_Announce_Interval = 30 
Min_Reconnect_Time = 20 
Request_Timeout = 10 
Smooth_Connects = False 
Send_Buffer_Watermark_MB = 5  
Seed_Time_Limit = 60 
Tracker_Completion_Timeout = 30 
Upload_Rate_Limit_KB = 0 #@param {type:"integer"}
User_Agent = '"Utorrent"/3.5.5' 

sett = {
    'allow_multiple_connections_per_ip': True,
    'active_downloads': Active_Downloads,
    'active_seeds':Active_Seeds,
    'active_checking': Active_Checking,
    'active_limit':Active_Limit,
    'active_seeds':Active_Seeds,    
    'announce_to_all_tiers': Announce_To_All_Tiers,
    'announce_to_all_trackers':Announce_To_All_Trackers,
    'allow_multiple_connections_per_ip':True,
    'auto_sequential':True,
    'auto_manage_interval': 30,
    'connections_limit':Connections_Limit,
    'connection_speed': Connection_Speed,
    'connections_slack':Connections_Slack,
    'close_redundant_connections':True,
    'file_pool_size':File_Pool_Size,
    'local_service_announce_interval':200,
    'max_rejects':Max_Rejects,
    'max_peerlist_size': Max_Peerlist_Size,
    'max_paused_peerlist_size': Max_Paused_Peerlist_Size,
    'max_failcount':Max_Failcount,
    'min_announce_interval':Min_Announce_Interval,
    'min_reconnect_time': Min_Reconnect_Time,
    'peer_timeout':20,
    'request_timeout':Request_Timeout,
    'half_open_limit': Half_Open_Limit,
    'send_buffer_watermark':Send_Buffer_Watermark_MB*1024*1024,
    'smooth_connects':Smooth_Connects,
    'strict_end_game_mode':True,
    'seed_time_limit': Seed_Time_Limit,
    'tracker_completion_timeout':Tracker_Completion_Timeout,
    'upload_rate_limit':Upload_Rate_Limit_KB*1024,
    'user_agent': User_Agent,
    #'inactivity_timeout':20,
    #'ban_web_seeds':False,
    }

ses.apply_settings(sett)
print("Client settings updated")

#'inactivity_timeout':20,
#'ban_web_seeds':False,
  
settings = ses.get_settings()
setn =[]

for x in settings:
  setn.append(str(x))

setn.sort()

for x in setn:
  print("'{0}': {1},".format(x,settings[x]))

#### Load

In [ ]:

#@markdown 

Load_From = "" #@param ["", "/content/drive/Shareddrives/SM1/Downloads/.uncomplete", "/content/drive/Shareddrives/SM2/Downloads/.uncomplete", "/content/drive/Shareddrives/SM3/Downloads/.uncomplete", "/content/drive/Shareddrives/SM4/Downloads/.uncomplete"]
Local = False  #@param {type:"boolean"}
Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Category = "Movies" #@param ["Movies", "TV", "Apps", "Games"]
Sort = "By Seeders"  #@param ["By Seeders", "By Time", "By Size"]
Order = "Descending" #@param ["Descending", "Ascending"]
Backup = True  #@param {type:"boolean"}

sort = l337x_seeders
if Sort == "By Time":
  if Order == "Descending":
    sort = l337x_time_desc
  else:
    sort = l337x_time_asc
elif Sort == "By Size":
  if Order == "Descending":
    sort = l337x_size_desc
  else:
    sort = l337x_size_asc

def load_uncompleted(path, local = False):
  content = next(os.walk(path))[1]
  if content:
    print(" Choose a directory to load:")
    for i,c in enumerate(content[:]):
      print("   {0}- {1}".format(i+1, c))
    print()
    choice = input().lower()
    if "" == choice or " " == choice or  "q" == choice or "s" == choice:
      return
    elif "," in choice:
      choices = choice.split(",")
    else:
      choices = [choice]
    for select in choices:
      select = int(select) - 1
      if select >= len(content) or select < 0 :
        print("\nThe choice is out of range.\n")
      else:
        if not local:
          print("Loading {0} into {1}\n".format(content[select], temp_save))
          p0 = path + "/" + content[select]
          !cp -r "$p0" "$temp_save"
          print("{0} is Loaded\n".format(content[select]))
        r = resume_download(content[select])
        if r == -2:
          break
        print("\n   ...................\n          Done\n")
  else:
      print("Chosen directory is empty.")

def resume_download(item):
  item = name_movie(item)
  print(f"Downloading: {item}\nSelect a source:\n 1 - 1337x\n 2 - rarbg\n")
  select = input("").lower()
  if select == "s" or select == "":
    return -1
  elif select == "q" or select == " ":
    return -2
  elif select == "1":
    search("1337x", item, category = Category, Type = Type, sort1337x = sort, backup = Backup, uncomplete_source = p0)
  elif select == "2":
    search("rarbg", item,  category = Category, Type = Type, backup = Backup, auto = True, uncomplete_source = p0)
  return 0

if Load_From:
  load_uncompleted(Load_From)
elif Local:
  load_uncompleted(temp_save, True)


#### Add

In [ ]:

#@markdown

from lib2to3.pytree import type_repr
from google.colab import files
Source = "Local" #@param ["Local", "Upload File", "Magnet link"] {type:"string"}
Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Replace = False #@param {type:"boolean"}
Sequential_Download = False #@param{type:"boolean"}

attr = attrs()
attr.type_ = Type
attr.replace = Replace
attr.sequential = Sequential_Download
if Source == "Local" or Source == "Upload File":
  nms = []
  if Source == "Upload File":
    source = files.upload()
    nms = list(source.keys())
  add_local_files(attr)

if Source == "Magnet link":
  magnet_link = ""
  while True:
    magnet = input("Enter magnet link or type q:  ").lower()
    if magnet_link == "q":
      break
    name = get_magnet_name(magnet_link)
    if Type == Types.Movie and movie_exists(name) and not Replace:
      print("Fetching local files:    " + name + " is already downloaded.")
      continue

    if Type == Types.Episode and episode_exists(name) and not Replace:
      print("Fetching local files:    " + name + " is already downloaded.")
      continue

    if Type == Types.Season and season_exists(name) and not Replace:
      print("Fetching local files:    " + name + " is already downloaded.")
      continue
    
    attr.name = name
    add_torrent_magnet(magnet, attr)   


#### Metadata

In [ ]:

#@markdown 

print("Downloading metadata")
checks=[]
for h in active:
  h.resume()
  checks.append(h)

while (checks):
  i = input("enter anything to exit  ")
  if i :
    break
    
  for h in checks:
    if h.has_metadata() :
      h.pause()
      checks.remove(h)
      print("Enter anything to quit.")
  time.sleep(1)

for h in active:
  if(h.has_metadata()):
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    for x in range(files.num_files()):
      h.file_priority(x,5)

time.sleep(2)

#@markdown

total_download = 0

print("Download files:")
for h in active:

  print("")
  print("  "+h.name())

  if(h.has_metadata()):

    priorities = h.file_priorities()
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    file_sizes = tor_info.total_size()
    total_download += file_sizes
    size_str = data_str(file_sizes)

    print("  Total download size: " + size_str)
    print("  Index Size    Name")

    fi_str=""
    for x in range(files.num_files()):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))
  else:
    print("  Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print(
  
filterfiles()

#### Magnet

In [ ]:

#@markdown

Magnet_Link = "" #@param {type:"string"}
Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Sequential_Download = False #@param{type:"boolean"}

params = {"save_path": temp_save}
name = get_magnet_name(Magnet_Link)
attr = attrs()
attr.type_ = Type
attr.name = name
attr.sequential = Sequential_Download
add_torrent_magnet(Magnet_Link, attr)

#### List Files

In [ ]:

#@markdown

total_download = 0

count_str = str(len(active))

print("Downloads: {0}".format(count_str))
for handle in active:
  print("")
  print("  "+handle.name())

  if(handle.has_metadata()):
    priorities = handle.file_priorities()
    torinfo = handle.get_torrent_info()
    files = torinfo.files()
    status= handle.status()
    file_sizes = status.total_wanted
    total_download+=file_sizes

    size_str = data_str(file_sizes)

    print("   Total download size: " + size_str)
    print("   Index Size    Name")

    fi_str=""
    for x in range(len(priorities)):
      if (priorities[x]>0):
        fs_str= data_str(files.file_size(x))
        fi_str = "   {:3d}-  ".format(x) + fs_str

        print(fi_str,files.file_name(x))
  else:
    print("    Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)

#### Status

In [ ]:

#@markdown 

from IPython.display import display
import ipywidgets as widgets

downloads= ses.get_torrents()

session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

for index, download in enumerate(downloads[:]):
    bar = download_bars[index]
    shift = 0
    try:
      status = download.status()
      state = ""
      if status.paused == True:
        state = "PAUSED"
      else:
        state_str[status.state]
      bar.description = " ".join(
          [
              download.name(),
              speed_str(ByteToMB(status.download_rate)),
              state,
              "{0} / {1}".format(status.total_wanted_done, status.total_wanted)
          ]
      )
      bar.value = round(status.progress * 100,2)
    except:
      downloads.remove(download)
      download_bars.remove(index-shift)
      shift = shift + 1
      print("Invalid torrent")
      continue


In [ ]:

#@markdown

for a in active:
  print(f"{a.name()}: \n")
  dump(a.attr)
  print("\n\n")

#### Trackers

In [ ]:

#@markdown 

new_trackers = ["udp://tracker.opentrackr.org:1337/announce", "udp://tracker.internetwarriors.net:1337/announce", "udp://exodus.desync.com:6969/announce", "udp://tracker.cyberia.is:6969/announce", "udp://3rt.tace.ru:60889/announce", "http://5rt.tace.ru:60889/announce", "udp://explodie.org:6969/announce", "udp://47.ip-51-68-199.eu:6969/announce", "udp://opentracker.i2p.rocks:6969/announce", "http://open.acgnxtracker.com:80/announce", "udp://www.torrent.eu.org:451/announce", "udp://tracker.torrent.eu.org:451/announce", "udp://tracker.tiny-vps.com:6969/announce", "udp://tracker.ds.is:6969/announce", "udp://retracker.lanta-net.ru:2710/announce", "udp://open.stealth.si:80/announce", "udp://tracker.moeking.me:6969/announce", "udp://ipv4.tracker.harry.lu:80/announce", "http://tracker.dler.org:6969/announce", "http://rt.tace.ru:80/announce", "udp://93.158.213.92:1337/announce", "udp://138.255.103.83:1337/announce", "udp://208.83.20.20:6969/announce", "udp://193.218.118.220:6969/announce", "udp://79.111.118.59:60889/announce", "http://93.157.234.32:60889/announce", "udp://184.105.151.164:6969/announce", "udp://51.68.199.47:6969/announce", "udp://51.81.46.170:6969/announce", "udp://91.216.110.52:451/announce", "udp://89.234.156.205:451/announce", "udp://5.206.60.196:6969/announce", "udp://5.226.148.20:6969/announce", "udp://37.235.174.46:2710/announce", "udp://185.181.60.67:80/announce", "udp://138.201.150.56:6969/announce", "udp://51.15.40.114:80/announce", "http://168.119.237.9:6969/announce", "http://195.201.31.194:80/announce", "https://trakx.herokuapp.com:443/announce", "https://w.wwwww.wtf:443/announce", "https://tracker.tamersunion.org:443/announce", "https://tracker.sloppyta.co:443/announce", "https://tracker.renfei.net:443/announce", "https://tracker.nitrix.me:443/announce",
                "https://tracker.nanoha.org:443/announce", "https://tracker.lelux.fi:443/announce", "https://tracker.imgoingto.icu:443/announce", "https://tracker.hama3.net:443/announce", "https://tracker.foreverpirates.co:443/announce", "http://tracker2.wasabii.com.tw:6969/announce", "udp://tracker.sktorrent.net:6969/announce", "http://www.wareztorrent.com:80/announce", "udp://bt.xxx-tracker.com:2710/announce", "udp://tracker.eddie4.nl:6969/announce", "udp://tracker.grepler.com:6969/announce", "udp://tracker.mg64.net:2710/announce", "udp://wambo.club:1337/announce", "udp://tracker.dutchtracking.com:6969/announce", "udp://tc.animereactor.ru:8082/announce", "udp://tracker.justseed.it:1337/announce", "udp://tracker.leechers-paradise.org:6969/announce", "https://open.kickasstracker.com:443/announce", "udp://tracker.coppersurfer.tk:6969/announce", "http://87.253.152.137/announce", "http://91.216.110.47/announce", "http://91.217.91.21:3218/announce", "http://91.218.230.81:6969/announce", "http://93.92.64.5/announce", "http://atrack.pow7.com/announce", "http://bt.henbt.com:2710/announce", "http://bt.pusacg.org:8080/announce", "https://tracker.bt-hash.com:443/announce", "udp://tracker.leechers-paradise.org:6969", "https://182.176.139.129:6969/announce", "udp://zephir.monocul.us:6969/announce", "https://tracker.dutchtracking.com:80/announce", "https://grifon.info:80/announce", "udp://tracker.kicks-ass.net:80/announce", "udp://p4p.arenabg.com:1337/announce", "udp://tracker.aletorrenty.pl:2710/announce", "https://tracker.parrotsec.org:443/announce", "https://tracker.moxing.party:6969/announce", "https://tracker.ipv6tracker.ru:80/announce", "https://tracker.fastdownload.xyz:443/announce", "https://gwp2-v19.rinet.ru:80/announce", "https://tr.kxmp.cf:80/announce", "https://explodie.org:6969/announce"]

for handle in active:
  last_tier = handle.trackers()[-1]['tier']
  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': last_tier}
    handle.add_tracker(annouce_entry) 

#### List Trackers

In [ ]:

#@markdown 

handle = active[-1]
trackers = handle.trackers()
print("Trackers count ", len(trackers))
for x in trackers:
  #print(x)
  print(x['url'], x['tier'])

#### Speed

In [ ]:

#@markdown

!sudo apt install speedtest-cli
!speedtest-cli

#### Verify

In [ ]:

#@markdown

downloads= ses.get_torrents()
not_copied=[]
for download in downloads:
    name = download.name()
    print("  Checking: {0}".format(name) )
    status = download.status()
    byte_check = status.total_wanted - status.total_wanted_done
    if download.is_finished() or byte_check == 0:
        copy_status = ""
        if movie_exists(name):
            copy_status ="copied"
        else:
            copy_status = "not copied"
            not_copied.append(download)
        print("  Status:   completed   {0}".format(copy_status))
    else:
        print("  Status:   not completed   state: {0}     progress: {1}%     downloaded: {2}     remained: {3}".format(status.state, round(status.progress*100,2), data_str(status.total_wanted_done), data_str(status.total_wanted-status.total_wanted_done)))
    print()
print(  "Downloads completed but not copied:")
if not not_copied:
    print("     None")
else:
    for index in range(len(not_copied)):
        print("     {0}- {1}".format( index + 1, not_copied[index].name()))
print()
if not_copied:
  print("Do you want to copy and complete saving these downloads? y/n")
  print()
  choice = input("")
  if choice == 'y':
    for handle in not_copied:
      newname = ""
      if Type == Types.Movie:
        newname = name_movie(name)
        save_path = Movies_Path
      elif Type == Types.Show:
        newname = name_show(name)
        save_path = Shows_Path
      elif Type == Types.Season:
        newname = name_season(name)
        if not newname:
          newname = name
        show = name_show(name)
        save_path= Shows_Path + "/" + show
        os.makedirs(save_path, exist_ok= True)
      elif Type == Types.Episode:
        newname = name_episode(name)
        show = name_show(name)
        season= name_season(name)
        if not season:
          season = name
        save_path= Shows_Path + "/" + show + "/" + season
        os.makedirs(save_path, exist_ok= True)
      else:
        save_path = drive_path
      status = handle.status()
      p1 = status.save_path + "/" +  handle.name()
      p2 = done_path + "/" + newname
      !mv "$p1" "$p2"
      if not newname:
        p2 = p2 + handle.name()
      !mv "$p2" "$save_path"
      print(".................")
      print("       Done")
